In [ ]:
# This mounts your Google Drive to the Colab VM.
# # TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs6353/assignments/assignment1/'
FOLDERNAME = 'cs6350/assignments/assignment1/'
bankFOLDERNAME = FOLDERNAME + '/bank/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
sys.path.append('/content/drive/My Drive/{}'.format(bankFOLDERNAME))

# This downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
# %cd /content/drive/My\ Drive/$FOLDERNAME/cs6350/
# !bash get_datasets.sh
%cd /content/drive/My\ Drive/$FOLDERNAME
%cd /content/drive/My\ Drive/$bankFOLDERNAME

# Install requirements from colab_requirements.txt
# TODO: Please change your path below to the colab_requirements.txt file
#! python -m pip install -r /content/drive/My\ Drive/$FOLDERNAME/colab_requirements.txt

Mounted at /content/drive
/content/drive/My Drive/cs6350/assignments/assignment1
/content/drive/My Drive/cs6350/assignments/assignment1/bank


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

def entropy_car(labels):
    counts = np.bincount(labels)
    probabilities = counts / len(labels)
    return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

def gini_index_car(labels):
    counts = np.bincount(labels)
    probabilities = counts / len(labels)
    return 1 - np.sum([p ** 2 for p in probabilities])

def majority_error_car(labels):
    majority_class_count = np.max(np.bincount(labels))
    return 1 - (majority_class_count / len(labels))

def weighted_majority_error_car(data, attribute, target):
    total_len = len(data)
    weighted_me = 0
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_me += weight * majority_error_car(subset[target])
    return weighted_me

def weighted_gini_car(data, attribute, target):
    total_len = len(data)
    weighted_gini = 0
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_gini += weight * gini_index_car(subset[target])
    return weighted_gini

def select_best_attribute_car(data, attributes, target, criterion):
    best_gain = -float('inf')
    best_attribute = None
    for attribute in attributes:
        if criterion == 'entropy':
            gain = information_gain_car(data, attribute, target)
        elif criterion == 'gini':
            gain = gini_index_car(data[target]) - weighted_gini_car(data, attribute, target)
        elif criterion == 'majority_error':
            gain = majority_error_car(data[target]) - weighted_majority_error_car(data, attribute, target)
        if gain > best_gain:
            best_gain = gain
            best_attribute = attribute
    return best_attribute

def information_gain_car(data, attribute, target):
    total_entropy = entropy_car(data[target])
    weighted_sum = 0
    total_len = len(data)
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_sum += weight * entropy_car(subset[target])
    return total_entropy - weighted_sum

def id3_car(data, attributes, target, depth, max_depth, criterion):
    labels = data[target]

    if len(np.unique(labels)) == 1:
        return labels.iloc[0]
    if len(attributes) == 0 or depth == max_depth:
        return Counter(labels).most_common(1)[0][0]

    best_attribute = select_best_attribute_car(data, attributes, target, criterion)
    tree = {best_attribute: {}}

    for value in data[best_attribute].unique():
        subset = data[data[best_attribute] == value]
        if subset.empty:
            tree[best_attribute][value] = Counter(labels).most_common(1)[0][0]
        else:
            subtree = id3_car(subset, [attr for attr in attributes if attr != best_attribute], target, depth+1, max_depth, criterion)
            tree[best_attribute][value] = subtree
    return tree

def predict_car(tree, example):
    if not isinstance(tree, dict):
        return tree
    attribute = next(iter(tree))
    value = example[attribute]
    if value in tree[attribute]:
        return predict_car(tree[attribute][value], example)
    else:
        return Counter([predict_car(subtree, example) for subtree in tree[attribute].values()]).most_common(1)[0][0]

def calculate_error_car(tree, data):
    predictions = data.apply(lambda x: predict_car(tree, x), axis=1)
    incorrect = np.sum(predictions != data['label'])
    return incorrect / len(data)

def run_decision_tree_car(depths, train_data, test_data, criterion):
    train_errors = []
    test_errors = []

    for depth in depths:
        tree = id3_car(train_data, ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'], 'label', 0, depth, criterion)
        train_error = calculate_error_car(tree, train_data)
        test_error = calculate_error_car(tree, test_data)
        train_errors.append(train_error)
        test_errors.append(test_error)

    return train_errors, test_errors

def experiment():
    column_names_car = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'label']

    train_data_car = pd.read_csv("bank/train.csv", header=None, names=column_names_car)
    test_data_car = pd.read_csv("bank/test.csv", header=None, names=column_names_car)

    label_mapping = {'unacc': 0, 'acc': 1, 'good': 2, 'vgood': 3}
    train_data_car['label'] = train_data_car['label'].map(label_mapping)
    test_data_car['label'] = test_data_car['label'].map(label_mapping)

    depth = int(input("Enter the maximum tree depth: "))
    depths = range(1, depth + 1)

    train_car_ig, test_car_ig = run_decision_tree_car(depths, train_data_car, test_data_car, 'entropy')
    train_car_me, test_car_me = run_decision_tree_car(depths, train_data_car, test_data_car, 'majority_error')
    train_car_gini, test_car_gini = run_decision_tree_car(depths, train_data_car, test_data_car, 'gini')

    print("\nResults Summary:")
    print(f"\n{'Depth':<6} {'I.G(Train)':<12} {'I.G(Test)':<12} {'M.E(Train)':<12} {'M.E(Test)':<12} {'Gini(Train)':<12} {'Gini(Test)':<12}")
    for i in range(len(depths)):
        print(f"{depths[i]:<6} {train_car_ig[i]:<12.3f} {test_car_ig[i]:<12.3f} {train_car_me[i]:<12.3f} {test_car_me[i]:<12.3f} {train_car_gini[i]:<12.3f} {test_car_gini[i]:<12.3f}")

    min_car_ig_test = min(test_car_ig)
    min_car_me_test = min(test_car_me)
    min_car_gini_test = min(test_car_gini)

    print(f"\nLeast Information Gain error observed at depth {test_car_ig.index(min_car_ig_test) + 1} with test loss {min_car_ig_test:.3f}.")
    print(f"Least Majority Error observed at depth {test_car_me.index(min_car_me_test) + 1} with test loss {min_car_me_test:.3f}.")
    print(f"Least Gini error observed at depth {test_car_gini.index(min_car_gini_test) + 1} with test loss {min_car_gini_test:.3f}.")

if __name__ == "__main__":
    experiment()


Enter the maximum tree depth: 6

Results Summary:

Depth  I.G(Train)   I.G(Test)    M.E(Train)   M.E(Test)    Gini(Train)  Gini(Test)  
1      0.302        0.297        0.302        0.297        0.302        0.297       
2      0.222        0.223        0.301        0.316        0.222        0.223       
3      0.181        0.196        0.242        0.262        0.176        0.184       
4      0.082        0.147        0.130        0.243        0.089        0.133       
5      0.027        0.092        0.043        0.170        0.027        0.092       
6      0.000        0.093        0.000        0.170        0.000        0.093       

Least Information Gain error observed at depth 5 with test loss 0.092.
Least Majority Error observed at depth 5 with test loss 0.170.
Least Gini error observed at depth 5 with test loss 0.092.


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

def entropy(labels):
    counts = np.bincount(labels)
    probabilities = counts / len(labels)
    return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

def gini_index(labels):
    counts = np.bincount(labels)
    probabilities = counts / len(labels)
    return 1 - np.sum([p ** 2 for p in probabilities])

def majority_error(labels):
    majority_class_count = np.max(np.bincount(labels))
    return 1 - (majority_class_count / len(labels))

def weighted_majority_error(data, attribute, target):
    total_len = len(data)
    weighted_me = 0
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_me += weight * majority_error(subset[target])
    return weighted_me

def weighted_gini(data, attribute, target):
    total_len = len(data)
    weighted_gini = 0
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_gini += weight * gini_index(subset[target])
    return weighted_gini

def select_best_attribute(data, attributes, target, criterion):
    best_gain = -float('inf')
    best_attribute = None
    for attribute in attributes:
        if criterion == 'entropy':
            gain = information_gain(data, attribute, target)
        elif criterion == 'gini':
            gain = gini_index(data[target]) - weighted_gini(data, attribute, target)
        elif criterion == 'majority_error':
            gain = majority_error(data[target]) - weighted_majority_error(data, attribute, target)
        if gain > best_gain:
            best_gain = gain
            best_attribute = attribute
    return best_attribute

def information_gain(data, attribute, target):
    total_entropy = entropy(data[target])
    weighted_sum = 0
    total_len = len(data)
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_sum += weight * entropy(subset[target])
    return total_entropy - weighted_sum

def id3(data, attributes, target, depth, max_depth, criterion):
    labels = data[target]

    if len(np.unique(labels)) == 1:
        return labels.iloc[0]
    if len(attributes) == 0 or depth == max_depth:
        return Counter(labels).most_common(1)[0][0]

    best_attribute = select_best_attribute(data, attributes, target, criterion)
    tree = {best_attribute: {}}

    for value in data[best_attribute].unique():
        subset = data[data[best_attribute] == value]
        if subset.empty:
            tree[best_attribute][value] = Counter(labels).most_common(1)[0][0]
        else:
            subtree = id3(subset, [attr for attr in attributes if attr != best_attribute], target, depth+1, max_depth, criterion)
            tree[best_attribute][value] = subtree
    return tree


# def predict(tree, example):
#     if not isinstance(tree, dict):
#         return tree
#     attribute = next(iter(tree))
#     value = example[attribute]
#     if value not in tree[attribute]:
#         return max(tree[attribute], key=lambda x: len(str(tree[attribute][x])))
#     return predict(tree[attribute][value], example)

def predict(tree, example):
    if not isinstance(tree, dict):
        return tree
    attribute = next(iter(tree))
    value = example[attribute]
    if value in tree[attribute]:
        return predict(tree[attribute][value], example)
    else:
        # Handle missing branches by voting the most common prediction
        return Counter([predict(subtree, example) for subtree in tree[attribute].values()]).most_common(1)[0][0]

def calculate_error(tree, data):
    predictions = data.apply(lambda x: predict(tree, x), axis=1)
    incorrect = np.sum(predictions != data['y'])
    return incorrect / len(data)

def run_decision_tree(depths, train_data, test_data, criterion):
    train_errors = []
    test_errors = []

    for depth in depths:
        tree = id3(train_data, train_data.columns[:-1], 'y', 0, depth, criterion)
        train_error = calculate_error(tree, train_data)
        test_error = calculate_error(tree, test_data)
        train_errors.append(train_error)
        test_errors.append(test_error)

    return train_errors, test_errors

def experiment():
    column_names = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

    # Load the bank dataset
    train_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", header=None, names=column_names)
    test_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/test.csv", header=None, names=column_names)

    # Treat 'unknown' as a valid attribute value without special handling
    # Map the 'y' label to binary values
    train_data['y'] = train_data['y'].map({'yes': 1, 'no': 0})
    test_data['y'] = test_data['y'].map({'yes': 1, 'no': 0})

    depths = range(1, 17)

    # Run the decision tree with three different criteria
    train_ig, test_ig = run_decision_tree(depths, train_data, test_data, 'entropy')
    train_me, test_me = run_decision_tree(depths, train_data, test_data, 'majority_error')
    train_gini, test_gini = run_decision_tree(depths, train_data, test_data, 'gini')

    # Print results
    print(f"\n{'Depth':<6} {'I.G(Train)':<12} {'I.G(Test)':<12} {'M.E(Train)':<12} {'M.E(Test)':<12} {'Gini(Train)':<12} {'Gini(Test)':<12}")
    for i in range(len(depths)):
        print(f"{depths[i]:<6} {train_ig[i]:<12.3f} {test_ig[i]:<12.3f} {train_me[i]:<12.3f} {test_me[i]:<12.3f} {train_gini[i]:<12.3f} {test_gini[i]:<12.3f}")

    # Find minimum test error for each criterion
    min_ig_test = min(test_ig)
    min_me_test = min(test_me)
    min_gini_test = min(test_gini)

    print(f"\nLeast Information Gain test error observed at depth {test_ig.index(min_ig_test) + 1} with error {min_ig_test:.3f}.")
    print(f"Least Majority Error test error observed at depth {test_me.index(min_me_test) + 1} with error {min_me_test:.3f}.")
    print(f"Least Gini test error observed at depth {test_gini.index(min_gini_test) + 1} with error {min_gini_test:.3f}.")

if __name__ == "__main__":
    experiment()



Depth  I.G(Train)   I.G(Test)    M.E(Train)   M.E(Test)    Gini(Train)  Gini(Test)  
1      0.072        0.162        0.072        0.162        0.072        0.162       
2      0.000        0.167        0.000        0.167        0.000        0.167       
3      0.000        0.167        0.000        0.167        0.000        0.167       
4      0.000        0.167        0.000        0.167        0.000        0.167       
5      0.000        0.167        0.000        0.167        0.000        0.167       
6      0.000        0.167        0.000        0.167        0.000        0.167       
7      0.000        0.167        0.000        0.167        0.000        0.167       
8      0.000        0.167        0.000        0.167        0.000        0.167       
9      0.000        0.167        0.000        0.167        0.000        0.167       
10     0.000        0.167        0.000        0.167        0.000        0.167       
11     0.000        0.167        0.000        0.167        0.000

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

# Helper functions for entropy, gini index, and majority error
def entropy(labels):
    counts = np.bincount(labels)
    probabilities = counts / len(labels)
    return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

def gini_index(labels):
    counts = np.bincount(labels)
    probabilities = counts / len(labels)
    return 1 - np.sum([p ** 2 for p in probabilities])

def majority_error(labels):
    majority_class_count = np.max(np.bincount(labels))
    return 1 - (majority_class_count / len(labels))

# Fill missing "unknown" values with the majority value of the attribute in the training set
def fill_missing_values(train_data, test_data):
    print("\nMajority values used for replacing 'unknown' in training set:")
    for column in train_data.columns:
        if train_data[column].dtype == object:  # Only for categorical attributes
            # Get the majority value excluding 'unknown'
            majority_value = train_data[column][train_data[column] != 'unknown'].mode()[0]
            print(f"{column}: {majority_value}")
            # Replace 'unknown' with the majority value in both training and testing sets
            train_data[column].replace('unknown', majority_value, inplace=True)
            test_data[column].replace('unknown', majority_value, inplace=True)

def verify_replacement(train_data, test_data):
    print("Checking for 'unknown' in training data:")
    print(train_data.apply(lambda x: (x == 'unknown').sum()))

    print("\nChecking for 'unknown' in test data:")
    print(test_data.apply(lambda x: (x == 'unknown').sum()))


# Weighted measures for gini and majority error
def weighted_majority_error(data, attribute, target):
    total_len = len(data)
    weighted_me = 0
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_me += weight * majority_error(subset[target])
    return weighted_me

def weighted_gini(data, attribute, target):
    total_len = len(data)
    weighted_gini = 0
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_gini += weight * gini_index(subset[target])
    return weighted_gini

# Selecting the best attribute to split on
def select_best_attribute(data, attributes, target, criterion):
    best_gain = -float('inf')
    best_attribute = None
    for attribute in attributes:
        if criterion == 'entropy':
            gain = information_gain(data, attribute, target)
        elif criterion == 'gini':
            gain = gini_index(data[target]) - weighted_gini(data, attribute, target)
        elif criterion == 'majority_error':
            gain = majority_error(data[target]) - weighted_majority_error(data, attribute, target)
        if gain > best_gain:
            best_gain = gain
            best_attribute = attribute
    # Log the best attribute chosen for split
    print(f"Best attribute to split on: {best_attribute} with gain: {best_gain}")
    return best_attribute

# Information gain calculation
def information_gain(data, attribute, target):
    total_entropy = entropy(data[target])
    weighted_sum = 0
    total_len = len(data)
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_sum += weight * entropy(subset[target])
    return total_entropy - weighted_sum

# ID3 algorithm for building the decision tree
def id3(data, attributes, target, depth, max_depth, criterion):
    labels = data[target]

    if len(np.unique(labels)) == 1:
        return labels.iloc[0]
    if len(attributes) == 0 or depth == max_depth:
        return Counter(labels).most_common(1)[0][0]

    best_attribute = select_best_attribute(data, attributes, target, criterion)
    tree = {best_attribute: {}}

    for value in data[best_attribute].unique():
        subset = data[data[best_attribute] == value]
        if subset.empty:
            tree[best_attribute][value] = Counter(labels).most_common(1)[0][0]
        else:
            subtree = id3(subset, [attr for attr in attributes if attr != best_attribute], target, depth+1, max_depth, criterion)
            tree[best_attribute][value] = subtree
    return tree

# Function for making predictions with the decision tree
def predict(tree, example):
    if not isinstance(tree, dict):
        return tree
    attribute = next(iter(tree))
    value = example[attribute]
    if value in tree[attribute]:
        return predict(tree[attribute][value], example)
    else:
        # Handle missing branches by voting the most common prediction
        return Counter([predict(subtree, example) for subtree in tree[attribute].values()]).most_common(1)[0][0]

# Function to calculate error rate
def calculate_error(tree, data):
    predictions = data.apply(lambda x: predict(tree, x), axis=1)
    incorrect = np.sum(predictions != data['y'])
    return incorrect / len(data)

# Function to run decision tree with varying depths
def run_decision_tree(depths, train_data, test_data, criterion):
    train_errors = []
    test_errors = []

    for depth in depths:
        tree = id3(train_data, train_data.columns[:-1], 'y', 0, depth, criterion)
        train_error = calculate_error(tree, train_data)
        test_error = calculate_error(tree, test_data)
        train_errors.append(train_error)
        test_errors.append(test_error)

    return train_errors, test_errors

# Experiment function to run the entire process
def experiment():
    column_names = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

    # Load the bank dataset
    train_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", header=None, names=column_names)
    test_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/test.csv", header=None, names=column_names)

    # Fill missing 'unknown' values with the majority value
    fill_missing_values(train_data, test_data)

    # Map the 'y' label to binary values
    train_data['y'] = train_data['y'].map({'yes': 1, 'no': 0})
    test_data['y'] = test_data['y'].map({'yes': 1, 'no': 0})

    depths = range(1, 17)

    # Run the decision tree with three different criteria
    print("\nRunning Information Gain heuristic:")
    train_ig, test_ig = run_decision_tree(depths, train_data, test_data, 'entropy')

    print("\nRunning Majority Error heuristic:")
    train_me, test_me = run_decision_tree(depths, train_data, test_data, 'majority_error')

    print("\nRunning Gini Index heuristic:")
    train_gini, test_gini = run_decision_tree(depths, train_data, test_data, 'gini')

    # Print results
    print(f"\n{'Depth':<6} {'I.G(Train)':<12} {'I.G(Test)':<12} {'M.E(Train)':<12} {'M.E(Test)':<12} {'Gini(Train)':<12} {'Gini(Test)':<12}")
    for i in range(len(depths)):
        print(f"{depths[i]:<6} {train_ig[i]:<12.3f} {test_ig[i]:<12.3f} {train_me[i]:<12.3f} {test_me[i]:<12.3f} {train_gini[i]:<12.3f} {test_gini[i]:<12.3f}")

    # Find minimum test error for each criterion
    min_ig_test = min(test_ig)
    min_me_test = min(test_me)
    min_gini_test = min(test_gini)

    print(f"\nLeast Information Gain test error observed at depth {test_ig.index(min_ig_test) + 1} with error {min_ig_test:.3f}.")
    print(f"Least Majority Error test error observed at depth {test_me.index(min_me_test) + 1} with error {min_me_test:.3f}.")
    print(f"Least Gini test error observed at depth {test_gini.index(min_gini_test) + 1} with error {min_gini_test:.3f}.")

if __name__ == "__main__":
    experiment()

Streaming output truncated to the last 5000 lines.
Best attribute to split on: day with gain: 0.4285714285714286
Best attribute to split on: age with gain: 0.33333333333333337
Best attribute to split on: age with gain: 0.33333333333333337
Best attribute to split on: age with gain: 0.16666666666666663
Best attribute to split on: age with gain: 0.5
Best attribute to split on: age with gain: 0.16666666666666663
Best attribute to split on: age with gain: 0.5
Best attribute to split on: age with gain: 0.1428571428571429
Best attribute to split on: age with gain: 0.25
Best attribute to split on: age with gain: 0.19999999999999996
Best attribute to split on: age with gain: 0.25
Best attribute to split on: age with gain: 0.1428571428571429
Best attribute to split on: month with gain: 0.11111111111111116
Best attribute to split on: age with gain: 0.19999999999999996
Best attribute to split on: age with gain: 0.2857142857142857
Best attribute to split on: age with gain: 0.33333333333333337
Best 

In [ ]:
def check_for_missing_values(data):
    print("Checking for 'unknown', NaN, and blank spaces in the dataset:")
    for column in data.columns:
        unknown_count = data[column].str.contains('unknown', case=False, na=False).sum() if data[column].dtype == object else 0
        nan_count = data[column].isna().sum()
        blank_count = (data[column] == '').sum() if data[column].dtype == object else 0
        print(f"{column}: 'unknown'={unknown_count}, NaN={nan_count}, Blank={blank_count}")

def experiment():
    column_names = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

    # Load the bank dataset
    train_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", header=None, names=column_names)
    test_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/test.csv", header=None, names=column_names)

    # Check for alternate missing value representations
    print("Training data:")
    check_for_missing_values(train_data)
    print("\nTest data:")
    check_for_missing_values(test_data)

    # Continue with the rest of the experiment if needed (replace "unknown", etc.)
    # fill_missing_values() should be updated accordingly if alternate missing values are detected.

if __name__ == "__main__":
    experiment()

Training data:
Checking for 'unknown', NaN, and blank spaces in the dataset:
age: 'unknown'=0, NaN=0, Blank=0
job: 'unknown'=32, NaN=0, Blank=0
marital: 'unknown'=0, NaN=0, Blank=0
education: 'unknown'=182, NaN=0, Blank=0
default: 'unknown'=0, NaN=0, Blank=0
balance: 'unknown'=0, NaN=0, Blank=0
housing: 'unknown'=0, NaN=0, Blank=0
loan: 'unknown'=0, NaN=0, Blank=0
contact: 'unknown'=1433, NaN=0, Blank=0
day: 'unknown'=0, NaN=0, Blank=0
month: 'unknown'=0, NaN=0, Blank=0
duration: 'unknown'=0, NaN=0, Blank=0
campaign: 'unknown'=0, NaN=0, Blank=0
pdays: 'unknown'=0, NaN=0, Blank=0
previous: 'unknown'=0, NaN=0, Blank=0
poutcome: 'unknown'=4135, NaN=0, Blank=0
y: 'unknown'=0, NaN=0, Blank=0

Test data:
Checking for 'unknown', NaN, and blank spaces in the dataset:
age: 'unknown'=0, NaN=0, Blank=0
job: 'unknown'=33, NaN=0, Blank=0
marital: 'unknown'=0, NaN=0, Blank=0
education: 'unknown'=214, NaN=0, Blank=0
default: 'unknown'=0, NaN=0, Blank=0
balance: 'unknown'=0, NaN=0, Blank=0
housing: 'u

In [ ]:
# Function to fill "unknown" values with the majority value of that attribute in the training set
def fill_missing_values(train_data, test_data):
    print("\nMajority values used for replacing 'unknown' in training set:")
    for column in train_data.columns:
        if train_data[column].dtype == object:  # Only for categorical attributes
            # Get the majority value excluding 'unknown'
            majority_value = train_data[column][train_data[column] != 'unknown'].mode()[0]
            print(f"{column}: {majority_value}")
            # Replace 'unknown' with the majority value in both training and testing sets
            train_data[column].replace('unknown', majority_value, inplace=True)
            test_data[column].replace('unknown', majority_value, inplace=True)

# Modified function to log which attribute is being split on
def select_best_attribute(data, attributes, target, criterion):
    best_gain = -float('inf')
    best_attribute = None
    for attribute in attributes:
        if criterion == 'entropy':
            gain = information_gain(data, attribute, target)
        elif criterion == 'gini':
            gain = gini_index(data[target]) - weighted_gini(data, attribute, target)
        elif criterion == 'majority_error':
            gain = majority_error(data[target]) - weighted_majority_error(data, attribute, target)
        if gain > best_gain:
            best_gain = gain
            best_attribute = attribute
    # Log the best attribute chosen for split
    print(f"Best attribute to split on: {best_attribute} with gain: {best_gain}")
    return best_attribute

def experiment():
    column_names = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

    # Load the bank dataset
    train_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", header=None, names=column_names)
    test_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/test.csv", header=None, names=column_names)

    # Check for alternate missing value representations
    print("Training data:")
    check_for_missing_values(train_data)
    print("\nTest data:")
    check_for_missing_values(test_data)

    # Continue with the rest of the experiment if needed (replace "unknown", etc.)
    # fill_missing_values() should be updated accordingly if alternate missing values are detected.

if __name__ == "__main__":
    experiment()

Training data:
Checking for 'unknown', NaN, and blank spaces in the dataset:
age: 'unknown'=0, NaN=0, Blank=0
job: 'unknown'=32, NaN=0, Blank=0
marital: 'unknown'=0, NaN=0, Blank=0
education: 'unknown'=182, NaN=0, Blank=0
default: 'unknown'=0, NaN=0, Blank=0
balance: 'unknown'=0, NaN=0, Blank=0
housing: 'unknown'=0, NaN=0, Blank=0
loan: 'unknown'=0, NaN=0, Blank=0
contact: 'unknown'=1433, NaN=0, Blank=0
day: 'unknown'=0, NaN=0, Blank=0
month: 'unknown'=0, NaN=0, Blank=0
duration: 'unknown'=0, NaN=0, Blank=0
campaign: 'unknown'=0, NaN=0, Blank=0
pdays: 'unknown'=0, NaN=0, Blank=0
previous: 'unknown'=0, NaN=0, Blank=0
poutcome: 'unknown'=4135, NaN=0, Blank=0
y: 'unknown'=0, NaN=0, Blank=0

Test data:
Checking for 'unknown', NaN, and blank spaces in the dataset:
age: 'unknown'=0, NaN=0, Blank=0
job: 'unknown'=33, NaN=0, Blank=0
marital: 'unknown'=0, NaN=0, Blank=0
education: 'unknown'=214, NaN=0, Blank=0
default: 'unknown'=0, NaN=0, Blank=0
balance: 'unknown'=0, NaN=0, Blank=0
housing: 'u

claude... 3_1

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

def entropy(labels):
    counts = Counter(labels)
    probabilities = [count / len(labels) for count in counts.values()]
    return -sum(p * np.log2(p) for p in probabilities if p > 0)

def gini_index(labels):
    counts = Counter(labels)
    probabilities = [count / len(labels) for count in counts.values()]
    return 1 - sum(p ** 2 for p in probabilities)

def majority_error(labels):
    counts = Counter(labels)
    majority_class_count = max(counts.values())
    return 1 - (majority_class_count / len(labels))

def information_gain(data, attribute, target, criterion):
    total_metric = globals()[criterion](data[target])
    weighted_sum = 0
    total_len = len(data)

    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_sum += weight * globals()[criterion](subset[target])

    return total_metric - weighted_sum

def select_best_attribute(data, attributes, target, criterion):
    best_gain = -float('inf')
    best_attribute = None

    for attribute in attributes:
        gain = information_gain(data, attribute, target, criterion)
        if gain > best_gain:
            best_gain = gain
            best_attribute = attribute

    return best_attribute

def id3(data, attributes, target, depth, max_depth, criterion):
    labels = data[target]

    if len(labels.unique()) == 1:
        return labels.iloc[0]
    if len(attributes) == 0 or depth == max_depth:
        return labels.mode().iloc[0]

    best_attribute = select_best_attribute(data, attributes, target, criterion)

    if best_attribute is None:
        return labels.mode().iloc[0]

    tree = {best_attribute: {}}

    for value in data[best_attribute].unique():
        subset = data[data[best_attribute] == value]
        if len(subset) == 0:
            tree[best_attribute][value] = labels.mode().iloc[0]
        else:
            subtree = id3(subset, [attr for attr in attributes if attr != best_attribute], target, depth+1, max_depth, criterion)
            tree[best_attribute][value] = subtree

    return tree

def predict(tree, example):
    if not isinstance(tree, dict):
        return tree
    attribute = next(iter(tree))
    value = example[attribute]
    if value not in tree[attribute]:
        return max(tree[attribute].values(), key=lambda x: str(x))
    return predict(tree[attribute][value], example)

def calculate_error(tree, data, target):
    predictions = data.apply(lambda x: predict(tree, x), axis=1)
    return np.mean(predictions != data[target])

def run_decision_tree(depths, train_data, test_data, attributes, target, criterion):
    train_errors = []
    test_errors = []

    for depth in depths:
        tree = id3(train_data, attributes, target, 0, depth, criterion)
        train_error = calculate_error(tree, train_data, target)
        test_error = calculate_error(tree, test_data, target)
        train_errors.append(train_error)
        test_errors.append(test_error)

    return train_errors, test_errors

def preprocess_data(data):
    # Categorical columns (including those with 'unknown' as a valid category)
    categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']
    for col in categorical_columns:
        data[col] = data[col].astype('category')

    # Numeric columns
    numeric_columns = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
    for col in numeric_columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    return data

def experiment():
    column_names = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

    train_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", header=None, names=column_names)
    test_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/test.csv", header=None, names=column_names)

    train_data = preprocess_data(train_data)
    test_data = preprocess_data(test_data)

    attributes = column_names[:-1]  # All columns except the target
    target = 'y'

    depths = range(1, 17)

    criteria = ['entropy', 'majority_error', 'gini_index']
    results = {}

    for criterion in criteria:
        train_errors, test_errors = run_decision_tree(depths, train_data, test_data, attributes, target, criterion)
        results[criterion] = {'train': train_errors, 'test': test_errors}

    print(f"{'Depth':<6} {'I.G(Train)':<12} {'I.G(Test)':<12} {'M.E(Train)':<12} {'M.E(Test)':<12} {'Gini(Train)':<12} {'Gini(Test)':<12}")
    for i in range(len(depths)):
        print(f"{depths[i]:<6} "
              f"{results['entropy']['train'][i]:<12.3f} {results['entropy']['test'][i]:<12.3f} "
              f"{results['majority_error']['train'][i]:<12.3f} {results['majority_error']['test'][i]:<12.3f} "
              f"{results['gini_index']['train'][i]:<12.3f} {results['gini_index']['test'][i]:<12.3f}")

    for criterion in criteria:
        min_test_error = min(results[criterion]['test'])
        min_depth = results[criterion]['test'].index(min_test_error) + 1
        print(f"\nLeast {criterion.replace('_', ' ').title()} error observed at depth {min_depth} with test error {min_test_error:.3f}.")

if __name__ == "__main__":
    experiment()

Depth  I.G(Train)   I.G(Test)    M.E(Train)   M.E(Test)    Gini(Train)  Gini(Test)  
1      0.072        0.358        0.072        0.358        0.072        0.358       
2      0.000        0.517        0.000        0.517        0.000        0.517       
3      0.000        0.522        0.000        0.522        0.000        0.522       
4      0.000        0.522        0.000        0.522        0.000        0.522       
5      0.000        0.522        0.000        0.522        0.000        0.522       
6      0.000        0.522        0.000        0.522        0.000        0.522       
7      0.000        0.522        0.000        0.522        0.000        0.522       
8      0.000        0.522        0.000        0.522        0.000        0.522       
9      0.000        0.522        0.000        0.522        0.000        0.522       
10     0.000        0.522        0.000        0.522        0.000        0.522       
11     0.000        0.522        0.000        0.522        0.000 

In [ ]:
clause 3_2


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

def entropy(labels):
    counts = Counter(labels)
    probabilities = [count / len(labels) for count in counts.values()]
    return -sum(p * np.log2(p) for p in probabilities if p > 0)

def gini_index(labels):
    counts = Counter(labels)
    probabilities = [count / len(labels) for count in counts.values()]
    return 1 - sum(p ** 2 for p in probabilities)

def majority_error(labels):
    counts = Counter(labels)
    majority_class_count = max(counts.values())
    return 1 - (majority_class_count / len(labels))

def information_gain(data, attribute, target, criterion):
    total_metric = globals()[criterion](data[target])
    weighted_sum = 0
    total_len = len(data)

    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_sum += weight * globals()[criterion](subset[target])

    return total_metric - weighted_sum

def select_best_attribute(data, attributes, target, criterion):
    best_gain = -float('inf')
    best_attribute = None

    for attribute in attributes:
        gain = information_gain(data, attribute, target, criterion)
        if gain > best_gain:
            best_gain = gain
            best_attribute = attribute

    return best_attribute

def id3(data, attributes, target, depth, max_depth, criterion):
    labels = data[target]

    if len(labels.unique()) == 1:
        return labels.iloc[0]
    if len(attributes) == 0 or depth == max_depth:
        return labels.mode().iloc[0]

    best_attribute = select_best_attribute(data, attributes, target, criterion)

    if best_attribute is None:
        return labels.mode().iloc[0]

    tree = {best_attribute: {}}

    for value in data[best_attribute].unique():
        subset = data[data[best_attribute] == value]
        if len(subset) == 0:
            tree[best_attribute][value] = labels.mode().iloc[0]
        else:
            subtree = id3(subset, [attr for attr in attributes if attr != best_attribute], target, depth+1, max_depth, criterion)
            tree[best_attribute][value] = subtree

    return tree

def predict(tree, example):
    if not isinstance(tree, dict):
        return tree
    attribute = next(iter(tree))
    value = example[attribute]
    if value not in tree[attribute]:
        return max(tree[attribute].values(), key=lambda x: str(x))
    return predict(tree[attribute][value], example)

def calculate_error(tree, data, target):
    predictions = data.apply(lambda x: predict(tree, x), axis=1)
    return np.mean(predictions != data[target])

def run_decision_tree(depths, train_data, test_data, attributes, target, criterion):
    train_errors = []
    test_errors = []

    for depth in depths:
        tree = id3(train_data, attributes, target, 0, depth, criterion)
        train_error = calculate_error(tree, train_data, target)
        test_error = calculate_error(tree, test_data, target)
        train_errors.append(train_error)
        test_errors.append(test_error)

    return train_errors, test_errors

def replace_unknown_with_majority(data, categorical_columns):
    for col in categorical_columns:
        majority_value = data[col].mode().iloc[0]
        data[col] = data[col].replace('unknown', majority_value)
    return data

def preprocess_data(data, is_train=False, train_data=None):
    # Categorical columns (including those with 'unknown' as a valid category)
    categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']

    if is_train:
        data = replace_unknown_with_majority(data, categorical_columns)
    else:
        for col in categorical_columns:
            majority_value = train_data[col].mode().iloc[0]
            data[col] = data[col].replace('unknown', majority_value)

    for col in categorical_columns:
        data[col] = data[col].astype('category')

    # Numeric columns
    numeric_columns = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
    for col in numeric_columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    return data

def experiment():
    column_names = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

    train_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", header=None, names=column_names)
    test_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/test.csv", header=None, names=column_names)

    train_data = preprocess_data(train_data, is_train=True)
    test_data = preprocess_data(test_data, is_train=False, train_data=train_data)

    attributes = column_names[:-1]  # All columns except the target
    target = 'y'

    depths = range(1, 17)

    criteria = ['entropy', 'majority_error', 'gini_index']
    results = {}

    for criterion in criteria:
        train_errors, test_errors = run_decision_tree(depths, train_data, test_data, attributes, target, criterion)
        results[criterion] = {'train': train_errors, 'test': test_errors}

    print(f"{'Depth':<6} {'I.G(Train)':<12} {'I.G(Test)':<12} {'M.E(Train)':<12} {'M.E(Test)':<12} {'Gini(Train)':<12} {'Gini(Test)':<12}")
    for i in range(len(depths)):
        print(f"{depths[i]:<6} "
              f"{results['entropy']['train'][i]:<12.3f} {results['entropy']['test'][i]:<12.3f} "
              f"{results['majority_error']['train'][i]:<12.3f} {results['majority_error']['test'][i]:<12.3f} "
              f"{results['gini_index']['train'][i]:<12.3f} {results['gini_index']['test'][i]:<12.3f}")

    for criterion in criteria:
        min_test_error = min(results[criterion]['test'])
        min_depth = results[criterion]['test'].index(min_test_error) + 1
        print(f"\nLeast {criterion.replace('_', ' ').title()} error observed at depth {min_depth} with test error {min_test_error:.3f}.")

if __name__ == "__main__":
    experiment()

Depth  I.G(Train)   I.G(Test)    M.E(Train)   M.E(Test)    Gini(Train)  Gini(Test)  
1      0.072        0.358        0.072        0.358        0.072        0.358       
2      0.000        0.517        0.000        0.517        0.000        0.517       
3      0.000        0.522        0.000        0.522        0.000        0.522       
4      0.000        0.522        0.000        0.522        0.000        0.522       
5      0.000        0.522        0.000        0.522        0.000        0.522       
6      0.000        0.522        0.000        0.522        0.000        0.522       
7      0.000        0.522        0.000        0.522        0.000        0.522       
8      0.000        0.522        0.000        0.522        0.000        0.522       
9      0.000        0.522        0.000        0.522        0.000        0.522       
10     0.000        0.522        0.000        0.522        0.000        0.522       
11     0.000        0.522        0.000        0.522        0.000 

gpt bas 2.2


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

# Entropy Calculation
def entropy(labels):
    counts = np.bincount(labels)
    probabilities = counts / len(labels)
    return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

# Gini Index Calculation
def gini_index(labels):
    counts = np.bincount(labels)
    probabilities = counts / len(labels)
    return 1 - np.sum([p ** 2 for p in probabilities])

# Majority Error Calculation
def majority_error(labels):
    majority_class_count = np.max(np.bincount(labels))
    return 1 - (majority_class_count / len(labels))

# Weighted Majority Error for a Split
def weighted_majority_error(data, attribute, target):
    total_len = len(data)
    weighted_me = 0
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_me += weight * majority_error(subset[target])
    return weighted_me

# Weighted Gini Index for a Split
def weighted_gini(data, attribute, target):
    total_len = len(data)
    weighted_gini = 0
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_gini += weight * gini_index(subset[target])
    return weighted_gini

# Information Gain Calculation
def information_gain(data, attribute, target):
    total_entropy = entropy(data[target])
    weighted_sum = 0
    total_len = len(data)
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_sum += weight * entropy(subset[target])
    return total_entropy - weighted_sum

# Select Best Attribute Based on Criterion
def select_best_attribute(data, attributes, target, criterion):
    best_gain = -float('inf')
    best_attribute = None
    for attribute in attributes:
        if criterion == 'entropy':
            gain = information_gain(data, attribute, target)
        elif criterion == 'gini':
            gain = gini_index(data[target]) - weighted_gini(data, attribute, target)
        elif criterion == 'majority_error':
            gain = majority_error(data[target]) - weighted_majority_error(data, attribute, target)
        if gain > best_gain:
            best_gain = gain
            best_attribute = attribute
    return best_attribute

# ID3 Algorithm for Building the Tree
def id3(data, attributes, target, depth, max_depth, criterion):
    labels = data[target]

    if len(np.unique(labels)) == 1:
        return labels.iloc[0]
    if len(attributes) == 0 or depth == max_depth:
        return Counter(labels).most_common(1)[0][0]

    best_attribute = select_best_attribute(data, attributes, target, criterion)
    tree = {best_attribute: {}}

    for value in data[best_attribute].unique():
        subset = data[data[best_attribute] == value]
        if subset.empty:
            tree[best_attribute][value] = Counter(labels).most_common(1)[0][0]
        else:
            subtree = id3(subset, [attr for attr in attributes if attr != best_attribute], target, depth+1, max_depth, criterion)
            tree[best_attribute][value] = subtree
    return tree

# Prediction Function
def predict(tree, example):
    if not isinstance(tree, dict):
        return tree
    attribute = next(iter(tree))
    value = example[attribute]
    if value in tree[attribute]:
        return predict(tree[attribute][value], example)
    else:
        return Counter([predict(subtree, example) for subtree in tree[attribute].values()]).most_common(1)[0][0]

# Calculate Prediction Error
def calculate_error(tree, data):
    predictions = data.apply(lambda x: predict(tree, x), axis=1)
    incorrect = np.sum(predictions != data['y'])
    return incorrect / len(data)

# Running Decision Tree with Varying Depth
def run_decision_tree(depths, train_data, test_data, criterion):
    train_errors = []
    test_errors = []

    for depth in depths:
        tree = id3(train_data, train_data.columns[:-1], 'y', 0, depth, criterion)
        train_error = calculate_error(tree, train_data)
        test_error = calculate_error(tree, test_data)
        train_errors.append(train_error)
        test_errors.append(test_error)

    return train_errors, test_errors

# Experiment function for the Bank Marketing Dataset
def experiment():
    column_names = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan',
                    'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

    train_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", header=None, names=column_names)
    test_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/test.csv", header=None, names=column_names)

    label_mapping = {'yes': 1, 'no': 0}
    train_data['y'] = train_data['y'].map(label_mapping)
    test_data['y'] = test_data['y'].map(label_mapping)

    depths = range(1, 17)

    # Run for Information Gain, Majority Error, and Gini Index
    train_ig, test_ig = run_decision_tree(depths, train_data, test_data, 'entropy')
    train_me, test_me = run_decision_tree(depths, train_data, test_data, 'majority_error')
    train_gini, test_gini = run_decision_tree(depths, train_data, test_data, 'gini')

    # Reporting Results
    print(f"\n{'Depth':<6} {'I.G(Train)':<12} {'I.G(Test)':<12} {'M.E(Train)':<12} {'M.E(Test)':<12} {'Gini(Train)':<12} {'Gini(Test)':<12}")
    for i in range(len(depths)):
        print(f"{depths[i]:<6} {train_ig[i]:<12.3f} {test_ig[i]:<12.3f} {train_me[i]:<12.3f} {test_me[i]:<12.3f} {train_gini[i]:<12.3f} {test_gini[i]:<12.3f}")

    min_ig_test = min(test_ig)
    min_me_test = min(test_me)
    min_gini_test = min(test_gini)

    print(f"\nLowest Information Gain test error at depth {test_ig.index(min_ig_test) + 1}: {min_ig_test:.3f}")
    print(f"Lowest Majority Error test error at depth {test_me.index(min_me_test) + 1}: {min_me_test:.3f}")
    print(f"Lowest Gini Index test error at depth {test_gini.index(min_gini_test) + 1}: {min_gini_test:.3f}")

if __name__ == "__main__":
    experiment()



Depth  I.G(Train)   I.G(Test)    M.E(Train)   M.E(Test)    Gini(Train)  Gini(Test)  
1      0.072        0.162        0.072        0.162        0.072        0.162       
2      0.000        0.167        0.000        0.167        0.000        0.167       
3      0.000        0.167        0.000        0.167        0.000        0.167       
4      0.000        0.167        0.000        0.167        0.000        0.167       
5      0.000        0.167        0.000        0.167        0.000        0.167       
6      0.000        0.167        0.000        0.167        0.000        0.167       
7      0.000        0.167        0.000        0.167        0.000        0.167       
8      0.000        0.167        0.000        0.167        0.000        0.167       
9      0.000        0.167        0.000        0.167        0.000        0.167       
10     0.000        0.167        0.000        0.167        0.000        0.167       
11     0.000        0.167        0.000        0.167        0.000

gpt 3_2

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

# Entropy Calculation
def entropy(labels):
    counts = np.bincount(labels)
    probabilities = counts / len(labels)
    return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

# Gini Index Calculation
def gini_index(labels):
    counts = np.bincount(labels)
    probabilities = counts / len(labels)
    return 1 - np.sum([p ** 2 for p in probabilities])

# Majority Error Calculation
def majority_error(labels):
    majority_class_count = np.max(np.bincount(labels))
    return 1 - (majority_class_count / len(labels))

# Fill Missing Values ("unknown") with Majority Value in Training Set
def fill_missing_values(data, attributes):
    for attribute in attributes:
        majority_value = data[attribute][data[attribute] != 'unknown'].mode()[0]
        data[attribute] = data[attribute].replace('unknown', majority_value)

# Weighted Majority Error for a Split
def weighted_majority_error(data, attribute, target):
    total_len = len(data)
    weighted_me = 0
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_me += weight * majority_error(subset[target])
    return weighted_me

# Weighted Gini Index for a Split
def weighted_gini(data, attribute, target):
    total_len = len(data)
    weighted_gini = 0
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_gini += weight * gini_index(subset[target])
    return weighted_gini

# Information Gain Calculation
def information_gain(data, attribute, target):
    total_entropy = entropy(data[target])
    weighted_sum = 0
    total_len = len(data)
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_sum += weight * entropy(subset[target])
    return total_entropy - weighted_sum

# Select Best Attribute Based on Criterion
def select_best_attribute(data, attributes, target, criterion):
    best_gain = -float('inf')
    best_attribute = None
    for attribute in attributes:
        if criterion == 'entropy':
            gain = information_gain(data, attribute, target)
        elif criterion == 'gini':
            gain = gini_index(data[target]) - weighted_gini(data, attribute, target)
        elif criterion == 'majority_error':
            gain = majority_error(data[target]) - weighted_majority_error(data, attribute, target)
        if gain > best_gain:
            best_gain = gain
            best_attribute = attribute
    return best_attribute

# ID3 Algorithm for Building the Tree
def id3(data, attributes, target, depth, max_depth, criterion):
    labels = data[target]

    if len(np.unique(labels)) == 1:
        return labels.iloc[0]
    if len(attributes) == 0 or depth == max_depth:
        return Counter(labels).most_common(1)[0][0]

    best_attribute = select_best_attribute(data, attributes, target, criterion)
    tree = {best_attribute: {}}

    for value in data[best_attribute].unique():
        subset = data[data[best_attribute] == value]
        if subset.empty:
            tree[best_attribute][value] = Counter(labels).most_common(1)[0][0]
        else:
            subtree = id3(subset, [attr for attr in attributes if attr != best_attribute], target, depth+1, max_depth, criterion)
            tree[best_attribute][value] = subtree
    return tree

# Prediction Function
def predict(tree, example):
    if not isinstance(tree, dict):
        return tree
    attribute = next(iter(tree))
    value = example[attribute]
    if value in tree[attribute]:
        return predict(tree[attribute][value], example)
    else:
        return Counter([predict(subtree, example) for subtree in tree[attribute].values()]).most_common(1)[0][0]

# Calculate Prediction Error
def calculate_error(tree, data):
    predictions = data.apply(lambda x: predict(tree, x), axis=1)
    incorrect = np.sum(predictions != data['y'])
    return incorrect / len(data)

# Running Decision Tree with Varying Depth
def run_decision_tree(depths, train_data, test_data, criterion):
    train_errors = []
    test_errors = []

    for depth in depths:
        tree = id3(train_data, train_data.columns[:-1], 'y', 0, depth, criterion)
        train_error = calculate_error(tree, train_data)
        test_error = calculate_error(tree, test_data)
        train_errors.append(train_error)
        test_errors.append(test_error)

    return train_errors, test_errors

# Experiment function for the Bank Marketing Dataset
def experiment():
    column_names = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan',
                    'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

    train_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", header=None, names=column_names)
    test_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/test.csv", header=None, names=column_names)

    # Map target labels 'yes' and 'no' to 1 and 0
    label_mapping = {'yes': 1, 'no': 0}
    train_data['y'] = train_data['y'].map(label_mapping)
    test_data['y'] = test_data['y'].map(label_mapping)

    # Fill missing values ("unknown") in training and test data
    fill_missing_values(train_data, train_data.columns[:-1])
    fill_missing_values(test_data, test_data.columns[:-1])

    depths = range(1, 17)

    # Run for Information Gain, Majority Error, and Gini Index
    train_ig, test_ig = run_decision_tree(depths, train_data, test_data, 'entropy')
    train_me, test_me = run_decision_tree(depths, train_data, test_data, 'majority_error')
    train_gini, test_gini = run_decision_tree(depths, train_data, test_data, 'gini')

    # Reporting Results
    print(f"\n{'Depth':<6} {'I.G(Train)':<12} {'I.G(Test)':<12} {'M.E(Train)':<12} {'M.E(Test)':<12} {'Gini(Train)':<12} {'Gini(Test)':<12}")
    for i in range(len(depths)):
        print(f"{depths[i]:<6} {train_ig[i]:<12.3f} {test_ig[i]:<12.3f} {train_me[i]:<12.3f} {test_me[i]:<12.3f} {train_gini[i]:<12.3f} {test_gini[i]:<12.3f}")

    min_ig_test = min(test_ig)
    min_me_test = min(test_me)
    min_gini_test = min(test_gini)

    print(f"\nLowest Information Gain test error at depth {test_ig.index(min_ig_test) + 1}: {min_ig_test:.3f}")
    print(f"Lowest Majority Error test error at depth {test_me.index(min_me_test) + 1}: {min_me_test:.3f}")
    print(f"Lowest Gini Index test error at depth {test_gini.index(min_gini_test) + 1}: {min_gini_test:.3f}")

if __name__ == "__main__":
    experiment()



Depth  I.G(Train)   I.G(Test)    M.E(Train)   M.E(Test)    Gini(Train)  Gini(Test)  
1      0.072        0.162        0.072        0.162        0.072        0.162       
2      0.000        0.167        0.000        0.167        0.000        0.167       
3      0.000        0.167        0.000        0.167        0.000        0.167       
4      0.000        0.167        0.000        0.167        0.000        0.167       
5      0.000        0.167        0.000        0.167        0.000        0.167       
6      0.000        0.167        0.000        0.167        0.000        0.167       
7      0.000        0.167        0.000        0.167        0.000        0.167       
8      0.000        0.167        0.000        0.167        0.000        0.167       
9      0.000        0.167        0.000        0.167        0.000        0.167       
10     0.000        0.167        0.000        0.167        0.000        0.167       
11     0.000        0.167        0.000        0.167        0.000

---------

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

# Helper functions for entropy, gini index, and majority error
def entropy(labels):
    counts = np.bincount(labels)
    probabilities = counts / len(labels)
    return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

def gini_index(labels):
    counts = np.bincount(labels)
    probabilities = counts / len(labels)
    return 1 - np.sum([p ** 2 for p in probabilities])

def majority_error(labels):
    majority_class_count = np.max(np.bincount(labels))
    return 1 - (majority_class_count / len(labels))

# Weighted measures for gini and majority error
def weighted_majority_error(data, attribute, target):
    total_len = len(data)
    weighted_me = 0
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_me += weight * majority_error(subset[target])
    return weighted_me

def weighted_gini(data, attribute, target):
    total_len = len(data)
    weighted_gini = 0
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_gini += weight * gini_index(subset[target])
    return weighted_gini

# Selecting the best attribute to split on
def select_best_attribute(data, attributes, target, criterion):
    best_gain = -float('inf')
    best_attribute = None
    for attribute in attributes:
        if criterion == 'entropy':
            gain = information_gain(data, attribute, target)
        elif criterion == 'gini':
            gain = gini_index(data[target]) - weighted_gini(data, attribute, target)
        elif criterion == 'majority_error':
            gain = majority_error(data[target]) - weighted_majority_error(data, attribute, target)
        if gain > best_gain:
            best_gain = gain
            best_attribute = attribute
    return best_attribute

# Information gain calculation
def information_gain(data, attribute, target):
    total_entropy = entropy(data[target])
    weighted_sum = 0
    total_len = len(data)
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_sum += weight * entropy(subset[target])
    return total_entropy - weighted_sum

# ID3 algorithm for building the decision tree
def id3(data, attributes, target, depth, max_depth, criterion):
    labels = data[target]

    if len(np.unique(labels)) == 1:
        return labels.iloc[0]
    if len(attributes) == 0 or depth == max_depth:
        return Counter(labels).most_common(1)[0][0]

    best_attribute = select_best_attribute(data, attributes, target, criterion)
    tree = {best_attribute: {}}

    for value in data[best_attribute].unique():
        subset = data[data[best_attribute] == value]
        if subset.empty:
            tree[best_attribute][value] = Counter(labels).most_common(1)[0][0]
        else:
            subtree = id3(subset, [attr for attr in attributes if attr != best_attribute], target, depth+1, max_depth, criterion)
            tree[best_attribute][value] = subtree
    return tree

# Function for making predictions with the decision tree
def predict(tree, example):
    if not isinstance(tree, dict):
        return tree
    attribute = next(iter(tree))
    value = example[attribute]
    if value in tree[attribute]:
        return predict(tree[attribute][value], example)
    else:
        # Handle missing branches by voting the most common prediction
        return Counter([predict(subtree, example) for subtree in tree[attribute].values()]).most_common(1)[0][0]

# Function to calculate error rate
def calculate_error(tree, data):
    predictions = data.apply(lambda x: predict(tree, x), axis=1)
    incorrect = np.sum(predictions != data['y'])
    return incorrect / len(data)

# Function to run decision tree with varying depths
def run_decision_tree(depths, train_data, test_data, criterion):
    train_errors = []
    test_errors = []

    for depth in depths:
        tree = id3(train_data, train_data.columns[:-1], 'y', 0, depth, criterion)
        train_error = calculate_error(tree, train_data)
        test_error = calculate_error(tree, test_data)
        train_errors.append(train_error)
        test_errors.append(test_error)

    return train_errors, test_errors

# Experiment function to run the entire process for "unknown" as a regular value
def experiment_regular_unknown():
    column_names = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

    # Load the bank dataset
    train_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", header=None, names=column_names)
    test_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/test.csv", header=None, names=column_names)

    # Map the 'y' label to binary values
    train_data['y'] = train_data['y'].map({'yes': 1, 'no': 0})
    test_data['y'] = test_data['y'].map({'yes': 1, 'no': 0})

    depths = range(1, 17)

    # Run the decision tree with three different criteria
    train_ig, test_ig = run_decision_tree(depths, train_data, test_data, 'entropy')
    train_me, test_me = run_decision_tree(depths, train_data, test_data, 'majority_error')
    train_gini, test_gini = run_decision_tree(depths, train_data, test_data, 'gini')

    # Print results
    print(f"\n{'Depth':<6} {'I.G(Train)':<12} {'I.G(Test)':<12} {'M.E(Train)':<12} {'M.E(Test)':<12} {'Gini(Train)':<12} {'Gini(Test)':<12}")
    for i in range(len(depths)):
        print(f"{depths[i]:<6} {train_ig[i]:<12.3f} {test_ig[i]:<12.3f} {train_me[i]:<12.3f} {test_me[i]:<12.3f} {train_gini[i]:<12.3f} {test_gini[i]:<12.3f}")

    # Find minimum test error for each criterion
    min_ig_test = min(test_ig)
    min_me_test = min(test_me)
    min_gini_test = min(test_gini)

    print(f"\nLeast Information Gain test error observed at depth {test_ig.index(min_ig_test) + 1} with error {min_ig_test:.3f}.")
    print(f"Least Majority Error test error observed at depth {test_me.index(min_me_test) + 1} with error {min_me_test:.3f}.")
    print(f"Least Gini test error observed at depth {test_gini.index(min_gini_test) + 1} with error {min_gini_test:.3f}.")

experiment_regular_unknown()


Depth  I.G(Train)   I.G(Test)    M.E(Train)   M.E(Test)    Gini(Train)  Gini(Test)  
1      0.072        0.162        0.072        0.162        0.072        0.162       
2      0.000        0.167        0.000        0.167        0.000        0.167       
3      0.000        0.167        0.000        0.167        0.000        0.167       
4      0.000        0.167        0.000        0.167        0.000        0.167       
5      0.000        0.167        0.000        0.167        0.000        0.167       
6      0.000        0.167        0.000        0.167        0.000        0.167       
7      0.000        0.167        0.000        0.167        0.000        0.167       
8      0.000        0.167        0.000        0.167        0.000        0.167       
9      0.000        0.167        0.000        0.167        0.000        0.167       
10     0.000        0.167        0.000        0.167        0.000        0.167       
11     0.000        0.167        0.000        0.167        0.000

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

def entropy_car(labels):
    counts = np.bincount(labels)
    probabilities = counts / len(labels)
    return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

def gini_index_car(labels):
    counts = np.bincount(labels)
    probabilities = counts / len(labels)
    return 1 - np.sum([p ** 2 for p in probabilities])

def majority_error_car(labels):
    majority_class_count = np.max(np.bincount(labels))
    return 1 - (majority_class_count / len(labels))

def weighted_majority_error_car(data, attribute, target):
    total_len = len(data)
    weighted_me = 0
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_me += weight * majority_error(subset[target])
    return weighted_me

def weighted_gini_car(data, attribute, target):
    total_len = len(data)
    weighted_gini = 0
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_gini += weight * gini_index(subset[target])
    return weighted_gini

def select_best_attribute_car(data, attributes, target, criterion):
    best_gain = -float('inf')
    best_attribute = None
    for attribute in attributes:
        if criterion == 'entropy':
            gain = information_gain(data, attribute, target)
        elif criterion == 'gini':
            gain = gini_index(data[target]) - weighted_gini(data, attribute, target)
        elif criterion == 'majority_error':
            gain = majority_error(data[target]) - weighted_majority_error(data, attribute, target)
        if gain > best_gain:
            best_gain = gain
            best_attribute = attribute
    return best_attribute

def information_gain_car(data, attribute, target):
    total_entropy = entropy(data[target])
    weighted_sum = 0
    total_len = len(data)
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_sum += weight * entropy(subset[target])
    return total_entropy - weighted_sum

def id3_car(data, attributes, target, depth, max_depth, criterion):
    labels = data[target]

    if len(np.unique(labels)) == 1:
        return labels.iloc[0]
    if len(attributes) == 0 or depth == max_depth:
        return Counter(labels).most_common(1)[0][0]

    best_attribute = select_best_attribute(data, attributes, target, criterion)
    tree = {best_attribute: {}}

    for value in data[best_attribute].unique():
        subset = data[data[best_attribute] == value]
        if subset.empty:
            tree[best_attribute][value] = Counter(labels).most_common(1)[0][0]
        else:
            subtree = id3(subset, [attr for attr in attributes if attr != best_attribute], target, depth+1, max_depth, criterion)
            tree[best_attribute][value] = subtree
    return tree

def predict_car(tree, example):
    if not isinstance(tree, dict):
        return tree
    attribute = next(iter(tree))
    value = example[attribute]
    if value in tree[attribute]:
        return predict(tree[attribute][value], example)
    else:
        return Counter([predict(subtree, example) for subtree in tree[attribute].values()]).most_common(1)[0][0]

def calculate_error_car(tree, data):
    predictions = data.apply(lambda x: predict(tree, x), axis=1)
    incorrect = np.sum(predictions != data['y'])
    return incorrect / len(data)

def run_decision_tree_car(depths, train_data, test_data, criterion):
    train_errors = []
    test_errors = []

    for depth in depths:
        tree = id3(train_data, ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome'], 'y', 0, depth, criterion)
        train_error = calculate_error(tree, train_data)
        test_error = calculate_error(tree, test_data)
        train_errors.append(train_error)
        test_errors.append(test_error)

    return train_errors, test_errors

# def run_decision_tree_car(depths, train_data, test_data, criterion):
#     train_errors = []
#     test_errors = []

#     for depth in depths:
#         tree = id3_car(train_data, ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'], 'label', 0, depth, criterion)
#         train_error = calculate_error_car(tree, train_data)
#         test_error = calculate_error_car(tree, test_data)
#         train_errors.append(train_error)
#         test_errors.append(test_error)

#     return train_errors, test_errors

def experiment():
    column_names = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

    # Load the bank dataset
    train_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", header=None, names=column_names)
    test_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/test.csv", header=None, names=column_names)

    label_mapping = {'yes': 1, 'no': 0}
    train_data['y'] = train_data['y'].map(label_mapping)
    test_data['y'] = test_data['y'].map(label_mapping)

    depths = range(1, 17)

    train_ig, test_ig = run_decision_tree_car(depths, train_data, test_data, 'entropy')
    train_me, test_me = run_decision_tree_car(depths, train_data, test_data, 'majority_error')
    train_gini, test_gini = run_decision_tree_car(depths, train_data, test_data, 'gini')

    print(f"\n{'Depth':<6} {'I.G(Train)':<12} {'I.G(Test)':<12} {'M.E(Train)':<12} {'M.E(Test)':<12} {'Gini(Train)':<12} {'Gini(Test)':<12}")
    for i in range(len(depths)):
        print(f"{depths[i]:<6} {train_ig[i]:<12.3f} {test_ig[i]:<12.3f} {train_me[i]:<12.3f} {test_me[i]:<12.3f} {train_gini[i]:<12.3f} {test_gini[i]:<12.3f}")

    min_ig_test = min(test_ig)
    min_me_test = min(test_me)
    min_gini_test = min(test_gini)

    print(f"\nLeast Information Gain error observed at depth {test_ig.index(min_ig_test) + 1} with test loss {min_ig_test:.3f}.")
    print(f"Least Majority Error observed at depth {test_me.index(min_me_test) + 1} with test loss {min_me_test:.3f}.")
    print(f"Least Gini error observed at depth {test_gini.index(min_gini_test) + 1} with test loss {min_gini_test:.3f}.")

if __name__ == "__main__":
    experiment()



Depth  I.G(Train)   I.G(Test)    M.E(Train)   M.E(Test)    Gini(Train)  Gini(Test)  
1      0.072        0.162        0.072        0.162        0.072        0.162       
2      0.000        0.167        0.000        0.167        0.000        0.167       
3      0.000        0.167        0.000        0.167        0.000        0.167       
4      0.000        0.167        0.000        0.167        0.000        0.167       
5      0.000        0.167        0.000        0.167        0.000        0.167       
6      0.000        0.167        0.000        0.167        0.000        0.167       
7      0.000        0.167        0.000        0.167        0.000        0.167       
8      0.000        0.167        0.000        0.167        0.000        0.167       
9      0.000        0.167        0.000        0.167        0.000        0.167       
10     0.000        0.167        0.000        0.167        0.000        0.167       
11     0.000        0.167        0.000        0.167        0.000

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

# Helper functions for entropy, gini index, and majority error
def entropy(labels):
    counts = np.bincount(labels)
    probabilities = counts / len(labels)
    return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

def gini_index(labels):
    counts = np.bincount(labels)
    probabilities = counts / len(labels)
    return 1 - np.sum([p ** 2 for p in probabilities])

def majority_error(labels):
    majority_class_count = np.max(np.bincount(labels))
    return 1 - (majority_class_count / len(labels))

# Fill missing "unknown" values with the majority value of the attribute in the training set
def fill_missing_values(train_data, test_data):
    for column in train_data.columns:
        if train_data[column].dtype == object:  # Only for categorical attributes
            majority_value = train_data[column][train_data[column] != 'unknown'].mode()[0]
            # Replace 'unknown' with the majority value in both training and testing sets
            train_data[column].replace('unknown', majority_value, inplace=True)
            test_data[column].replace('unknown', majority_value, inplace=True)

# Weighted measures for gini and majority error
def weighted_majority_error(data, attribute, target):
    total_len = len(data)
    weighted_me = 0
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_me += weight * majority_error(subset[target])
    return weighted_me

def weighted_gini(data, attribute, target):
    total_len = len(data)
    weighted_gini = 0
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_gini += weight * gini_index(subset[target])
    return weighted_gini

# Selecting the best attribute to split on
def select_best_attribute(data, attributes, target, criterion):
    best_gain = -float('inf')
    best_attribute = None
    for attribute in attributes:
        if criterion == 'entropy':
            gain = information_gain(data, attribute, target)
        elif criterion == 'gini':
            gain = gini_index(data[target]) - weighted_gini(data, attribute, target)
        elif criterion == 'majority_error':
            gain = majority_error(data[target]) - weighted_majority_error(data, attribute, target)
        if gain > best_gain:
            best_gain = gain
            best_attribute = attribute
    return best_attribute

# Information gain calculation
def information_gain(data, attribute, target):
    total_entropy = entropy(data[target])
    weighted_sum = 0
    total_len = len(data)
    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        weight = len(subset) / total_len
        weighted_sum += weight * entropy(subset[target])
    return total_entropy - weighted_sum

# ID3 algorithm for building the decision tree
def id3(data, attributes, target, depth, max_depth, criterion):
    labels = data[target]

    if len(np.unique(labels)) == 1:
        return labels.iloc[0]
    if len(attributes) == 0 or depth == max_depth:
        return Counter(labels).most_common(1)[0][0]

    best_attribute = select_best_attribute(data, attributes, target, criterion)
    tree = {best_attribute: {}}

    for value in data[best_attribute].unique():
        subset = data[data[best_attribute] == value]
        if subset.empty:
            tree[best_attribute][value] = Counter(labels).most_common(1)[0][0]
        else:
            subtree = id3(subset, [attr for attr in attributes if attr != best_attribute], target, depth+1, max_depth, criterion)
            tree[best_attribute][value] = subtree
    return tree

# Function for making predictions with the decision tree
def predict(tree, example):
    if not isinstance(tree, dict):
        return tree
    attribute = next(iter(tree))
    value = example[attribute]
    if value in tree[attribute]:
        return predict(tree[attribute][value], example)
    else:
        # Handle missing branches by voting the most common prediction
        return Counter([predict(subtree, example) for subtree in tree[attribute].values()]).most_common(1)[0][0]

# Function to calculate error rate
def calculate_error(tree, data):
    predictions = data.apply(lambda x: predict(tree, x), axis=1)
    incorrect = np.sum(predictions != data['y'])
    return incorrect / len(data)

# Function to run decision tree with varying depths
def run_decision_tree(depths, train_data, test_data, criterion):
    train_errors = []
    test_errors = []

    for depth in depths:
        tree = id3(train_data, train_data.columns[:-1], 'y', 0, depth, criterion)
        train_error = calculate_error(tree, train_data)
        test_error = calculate_error(tree, test_data)
        train_errors.append(train_error)
        test_errors.append(test_error)

    return train_errors, test_errors

def experiment():
    column_names = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

    # Load the bank dataset
    train_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", header=None, names=column_names)
    test_data = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/test.csv", header=None, names=column_names)

    # Fill missing 'unknown' values with the majority value
    fill_missing_values(train_data, test_data)

    # Map the 'y' label to binary values
    train_data['y'] = train_data['y'].map({'yes': 1, 'no': 0})
    test_data['y'] = test_data['y'].map({'yes': 1, 'no': 0})

    depths = range(1, 17)

    # Run the decision tree with three different criteria
    train_ig, test_ig = run_decision_tree(depths, train_data, test_data, 'entropy')
    train_me, test_me = run_decision_tree(depths, train_data, test_data, 'majority_error')
    train_gini, test_gini = run_decision_tree(depths, train_data, test_data, 'gini')

    # Print results
    print(f"\n{'Depth':<6} {'I.G(Train)':<12} {'I.G(Test)':<12} {'M.E(Train)':<12} {'M.E(Test)':<12} {'Gini(Train)':<12} {'Gini(Test)':<12}")
    for i in range(len(depths)):
        print(f"{depths[i]:<6} {train_ig[i]:<12.3f} {test_ig[i]:<12.3f} {train_me[i]:<12.3f} {test_me[i]:<12.3f} {train_gini[i]:<12.3f} {test_gini[i]:<12.3f}")

    # Find minimum test error for each criterion
    min_ig_test = min(test_ig)
    min_me_test = min(test_me)
    min_gini_test = min(test_gini)

    print(f"\nLeast Information Gain test error observed at depth {test_ig.index(min_ig_test) + 1} with error {min_ig_test:.3f}.")
    print(f"Least Majority Error test error observed at depth {test_me.index(min_me_test) + 1} with error {min_me_test:.3f}.")
    print(f"Least Gini test error observed at depth {test_gini.index(min_gini_test) + 1} with error {min_gini_test:.3f}.")

if __name__ == "__main__":
    experiment()



Depth  I.G(Train)   I.G(Test)    M.E(Train)   M.E(Test)    Gini(Train)  Gini(Test)  
1      0.072        0.162        0.072        0.162        0.072        0.162       
2      0.000        0.167        0.000        0.167        0.000        0.167       
3      0.000        0.167        0.000        0.167        0.000        0.167       
4      0.000        0.167        0.000        0.167        0.000        0.167       
5      0.000        0.167        0.000        0.167        0.000        0.167       
6      0.000        0.167        0.000        0.167        0.000        0.167       
7      0.000        0.167        0.000        0.167        0.000        0.167       
8      0.000        0.167        0.000        0.167        0.000        0.167       
9      0.000        0.167        0.000        0.167        0.000        0.167       
10     0.000        0.167        0.000        0.167        0.000        0.167       
11     0.000        0.167        0.000        0.167        0.000

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
from tabulate import tabulate

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

class DecisionTree:
    def __init__(self, max_depth=100, criterion='information_gain'):
        self.max_depth = max_depth
        self.criterion = criterion
        self.root = None

    def fit(self, X, y):
        self.n_features = X.shape[1]
        self.feature_types = self._determine_feature_types(X)
        self.root = self._grow_tree(X, y)

    # Function to determine if the attribute is of type categorical or numerical
    def _determine_feature_types(self, X):
        feature_types = []
        for i in range(X.shape[1]):
            unique_values = np.unique(X[:, i])
            if isinstance(unique_values[0], (int, float)) and len(unique_values) > 10:
                feature_types.append("numerical")
            else:
                feature_types.append("categorical")
        return feature_types

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_labels = len(y), len(np.unique(y))

        if depth >= self.max_depth or n_labels == 1:
            return Node(value=self._most_common_label(y))

        feat_idxs = np.arange(self.n_features)
        best_feat, best_value = self._best_split(X, y, feat_idxs)

        if best_feat is None:
            return Node(value=self._most_common_label(y))

        left_idxs, right_idxs = self._split(X, best_feat, best_value)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return Node(value=self._most_common_label(y))

        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)

        return Node(best_feat, best_value, left, right)

    def _best_split(self, X, y, feat_idxs):
        best_criteria = -np.inf
        split_idx, split_value = None, None

        for feat in feat_idxs:
            if self.feature_types[feat] == "numerical":
                values = X[:, feat]
                median = np.median(values)
                left_idxs = np.where(X[:, feat] <= median)[0]
                right_idxs = np.where(X[:, feat] > median)[0]
                if len(left_idxs) == 0 or len(right_idxs) == 0:
                    continue
                criteria_value = self._calc_criteria(y, left_idxs, right_idxs)
                if criteria_value > best_criteria:
                    best_criteria = criteria_value
                    split_idx = feat
                    split_value = median
            else:
                unique_values = np.unique(X[:, feat])
                for val in unique_values:
                    left_idxs, right_idxs = self._split(X, feat, val)
                    if len(left_idxs) == 0 or len(right_idxs) == 0:
                        continue
                    criteria_value = self._calc_criteria(y, left_idxs, right_idxs)
                    if criteria_value > best_criteria:
                        best_criteria = criteria_value
                        split_idx = feat
                        split_value = val

        if split_idx is None:
            return None, None

        return split_idx, split_value

    def _calc_criteria(self, y, left_idxs, right_idxs):
        left_y, right_y = y[left_idxs], y[right_idxs]
        if len(left_y) == 0 or len(right_y) == 0:
            return 0

        if self.criterion == 'information_gain':
            return self._information_gain(y, left_y, right_y)
        elif self.criterion == 'gini':
            return self._gini(y, left_y, right_y)
        elif self.criterion == 'majority_error':
            return self._majority_error(y, left_y, right_y)

    def _information_gain(self, y, left_y, right_y):
        n = len(y)
        parent_value = self._calculate_criteria_value(y)
        left_value = self._calculate_criteria_value(left_y)
        right_value = self._calculate_criteria_value(right_y)
        child_value = (len(left_y) / n) * left_value + (len(right_y) / n) * right_value
        return parent_value - child_value

    def _calculate_criteria_value(self, y):
        if self.criterion == 'information_gain':
            return self._entropy(y)
        elif self.criterion == 'gini':
            return self._gini_index(y)
        elif self.criterion == 'majority_error':
            return self._majority_error_value(y)

    def _gini(self, y, left_y, right_y):
        parent_gini = self._gini_index(y)
        gini_left = self._gini_index(left_y)
        gini_right = self._gini_index(right_y)
        n = len(y)
        child_gini = (len(left_y) / n) * gini_left + (len(right_y) / n) * gini_right
        return parent_gini - child_gini

    def _majority_error(self, y, left_y, right_y):
        parent_error = self._majority_error_value(y)
        majority_error_left = self._majority_error_value(left_y)
        majority_error_right = self._majority_error_value(right_y)
        n = len(y)
        child_error = (len(left_y) / n) * majority_error_left + (len(right_y) / n) * majority_error_right
        return parent_error - child_error

    def _split(self, X, feat, value):
        if self.feature_types[feat] == "numerical":
            left_idxs = np.where(X[:, feat] <= value)[0]
            right_idxs = np.where(X[:, feat] > value)[0]
        else:
            left_idxs = np.where(X[:, feat] == value)[0]
            right_idxs = np.where(X[:, feat] != value)[0]
        return left_idxs, right_idxs

    def _entropy(self, y):
        label_counts = Counter(y)
        total = len(y)
        entropy_value = 0.0

        for count in label_counts.values():
            p = count / total
            if p > 0:
                entropy_value -= p * np.log2(p)

        return entropy_value

    def _gini_index(self, y):
        counts = Counter(y)
        total = len(y)
        return 1.0 - sum((count / total) ** 2 for count in counts.values())

    def _majority_error_value(self, y):
        counts = Counter(y)
        total = len(y)
        return 1 - max(counts.values()) / total

    def _most_common_label(self, y):
        return Counter(y).most_common(1)[0][0]

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if self.feature_types[node.feature] == "numerical":
            if x[node.feature] <= node.threshold:
                return self._traverse_tree(x, node.left)
            else:
                return self._traverse_tree(x, node.right)
        else:
            if x[node.feature] == node.threshold:
                return self._traverse_tree(x, node.left)
            else:
                return self._traverse_tree(x, node.right)

column_headers = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'label']

df_train = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", names=column_headers)
df_test = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/test.csv", names=column_headers)

X_train = df_train.drop('label', axis=1).values
y_train = df_train['label'].values
X_test = df_test.drop('label', axis=1).values
y_test = df_test['label'].values

metrics = {crit: {'train': [], 'test': []} for crit in ['information_gain', 'majority_error', 'gini']}
best_depths = {crit: 0 for crit in metrics.keys()}
best_accs = {crit: 0 for crit in metrics.keys()}
max_depth = int(input("Enter maximum tree depth: "))

no_improve_counts = {crit: 0 for crit in metrics.keys()}
tolerance = 5

for depth in range(1, max_depth + 1):
    for criterion in metrics.keys():
        clf = DecisionTree(max_depth=depth, criterion=criterion)
        clf.fit(X_train, y_train)

        y_train_pred = clf.predict(X_train)
        y_test_pred = clf.predict(X_test)

        train_acc = np.mean(y_train == y_train_pred)
        test_acc = np.mean(y_test == y_test_pred)

        metrics[criterion]['train'].append(round(1 - train_acc, 3))
        metrics[criterion]['test'].append(round(1 - test_acc, 3))

        if test_acc > best_accs[criterion]:
            best_accs[criterion] = test_acc
            best_depths[criterion] = depth
            no_improve_counts[criterion] = 0
        else:
            no_improve_counts[criterion] += 1

table_data = []
for depth in range(1, len(metrics['information_gain']['train']) + 1):
    row = [depth,
           metrics['information_gain']['train'][depth - 1],
           metrics['information_gain']['test'][depth - 1],
           metrics['majority_error']['train'][depth - 1],
           metrics['majority_error']['test'][depth - 1],
           metrics['gini']['train'][depth - 1],
           metrics['gini']['test'][depth - 1]]

    if depth == best_depths['information_gain']:
        row = ['*' + str(depth),
               '*' + str(metrics['information_gain']['train'][depth - 1]),
               '*' + str(metrics['information_gain']['test'][depth - 1]),
               str(metrics['majority_error']['train'][depth - 1]),
               str(metrics['majority_error']['test'][depth - 1]),
               str(metrics['gini']['train'][depth - 1]),
               str(metrics['gini']['test'][depth - 1])]
    elif depth == best_depths['majority_error']:
        row = [str(depth),
               str(metrics['information_gain']['train'][depth - 1]),
               str(metrics['information_gain']['test'][depth - 1]),
               '*' + str(metrics['majority_error']['train'][depth - 1]),
               '*' + str(metrics['majority_error']['test'][depth - 1]),
               str(metrics['gini']['train'][depth - 1]),
               str(metrics['gini']['test'][depth - 1])]
    elif depth == best_depths['gini']:
        row = [str(depth),
               str(metrics['information_gain']['train'][depth - 1]),
               str(metrics['information_gain']['test'][depth - 1]),
               str(metrics['majority_error']['train'][depth - 1]),
               str(metrics['majority_error']['test'][depth - 1]),
               '*' + str(metrics['gini']['train'][depth - 1]),
               '*' + str(metrics['gini']['test'][depth - 1])]

    table_data.append(row)

headers = ["Depth", "I.G(Train)", "I.G(Test)", "M.E(Train)", "M.E(Test)", "Gini(Train)", "Gini(Test)"]
print(tabulate(table_data, headers=headers, tablefmt="pretty"))

for crit in ['information_gain', 'majority_error', 'gini']:
    print(f"Least {crit.replace('_', ' ').title()} error observed at depth {best_depths[crit]} with test loss {1-best_accs[crit]:.3f}.")

Enter maximum tree depth: 16
+-------+------------+-----------+------------+-----------+-------------+------------+
| Depth | I.G(Train) | I.G(Test) | M.E(Train) | M.E(Test) | Gini(Train) | Gini(Test) |
+-------+------------+-----------+------------+-----------+-------------+------------+
|   1   |   0.119    |   0.125   |   0.109    |   0.117   |    0.109    |   0.117    |
|   2   |   0.106    |   0.111   |   0.108    |   0.116   |    0.109    |   0.117    |
|   3   |   0.105    |   0.112   |   0.107    |   0.116   |    0.106    |   0.111    |
|   4   |   0.104    |   0.115   |   0.105    |   0.116   |    0.104    |   0.113    |
|   5   |   0.099    |   0.11    |   0.104    |   0.116   |    0.099    |   0.113    |
|  *6   |   *0.093   |  *0.109   |   0.102    |   0.116   |    0.09     |   0.111    |
|   7   |   0.089    |   0.111   |   *0.101   |  *0.115   |    0.085    |    0.11    |
|   8   |   0.085    |   0.111   |   0.098    |   0.116   |    0.08     |   0.111    |
|   9   |   0.

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
from tabulate import tabulate

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

class DecisionTree:
    def __init__(self, max_depth=100, criterion='information_gain'):
        self.max_depth = max_depth
        self.criterion = criterion
        self.root = None

    def fit(self, X, y):
        self.n_features = X.shape[1]
        self.feature_types = self._determine_feature_types(X)
        self.majority_values = self._impute_missing_values(X, y)
        X = self._replace_missing_values(X)
        self.root = self._grow_tree(X, y)

    def _determine_feature_types(self, X):
        feature_types = []
        for i in range(X.shape[1]):
            unique_values = np.unique(X[:, i])
            if isinstance(unique_values[0], (int, float)) and len(unique_values) > 10:
                feature_types.append("numerical")
            else:
                feature_types.append("categorical")
        return feature_types

    # Function to impute the missing (unknown) values with the most common values
    def _impute_missing_values(self, X, y):
        majority_values = {}
        for i in range(X.shape[1]):
            if self.feature_types[i] == "categorical":
                values = X[:, i]

                if 'unknown' in values:
                    majority_value = Counter(values[values != 'unknown']).most_common(1)[0][0]
                    majority_values[i] = majority_value
        return majority_values

    # Function to replace missing values (unknown) with the imputed values
    def _replace_missing_values(self, X):
        X_imputed = X.copy()
        for i in range(X.shape[1]):
            if i in self.majority_values:
                X_imputed[X[:, i] == 'unknown', i] = self.majority_values[i]
        return X_imputed

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_labels = len(y), len(np.unique(y))

        if depth >= self.max_depth or n_labels == 1:
            return Node(value=self._most_common_label(y))

        feat_idxs = np.arange(self.n_features)
        best_feat, best_value = self._best_split(X, y, feat_idxs)

        if best_feat is None:
            return Node(value=self._most_common_label(y))

        left_idxs, right_idxs = self._split(X, best_feat, best_value)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return Node(value=self._most_common_label(y))

        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)

        return Node(best_feat, best_value, left, right)

    def _best_split(self, X, y, feat_idxs):
        best_criteria = -np.inf
        split_idx, split_value = None, None

        for feat in feat_idxs:
            if self.feature_types[feat] == "numerical":
                values = X[:, feat]
                median = np.median(values.astype(float))
                left_idxs = np.where(values <= median)[0]
                right_idxs = np.where(values > median)[0]
                if len(left_idxs) == 0 or len(right_idxs) == 0:
                    continue
                criteria_value = self._calc_criteria(y, left_idxs, right_idxs)
                if criteria_value > best_criteria:
                    best_criteria = criteria_value
                    split_idx = feat
                    split_value = median
            else:
                unique_values = np.unique(X[:, feat])
                for val in unique_values:
                    if val == 'unknown':
                        continue
                    left_idxs, right_idxs = self._split(X, feat, val)
                    if len(left_idxs) == 0 or len(right_idxs) == 0:
                        continue
                    criteria_value = self._calc_criteria(y, left_idxs, right_idxs)
                    if criteria_value > best_criteria:
                        best_criteria = criteria_value
                        split_idx = feat
                        split_value = val

        if split_idx is None:
            return None, None

        return split_idx, split_value

    def _calc_criteria(self, y, left_idxs, right_idxs):
        left_y, right_y = y[left_idxs], y[right_idxs]
        if len(left_y) == 0 or len(right_y) == 0:
            return 0

        if self.criterion == 'information_gain':
            return self._information_gain(y, left_y, right_y)
        elif self.criterion == 'gini':
            return self._gini(y, left_y, right_y)
        elif self.criterion == 'majority_error':
            return self._majority_error(y, left_y, right_y)

    def _information_gain(self, y, left_y, right_y):
        n = len(y)
        parent_value = self._calculate_criteria_value(y)
        left_value = self._calculate_criteria_value(left_y)
        right_value = self._calculate_criteria_value(right_y)
        child_value = (len(left_y) / n) * left_value + (len(right_y) / n) * right_value
        return parent_value - child_value

    def _calculate_criteria_value(self, y):
        if self.criterion == 'information_gain':
            return self._entropy(y)
        elif self.criterion == 'gini':
            return self._gini_index(y)
        elif self.criterion == 'majority_error':
            return self._majority_error_value(y)

    def _gini(self, y, left_y, right_y):
        parent_gini = self._gini_index(y)
        gini_left = self._gini_index(left_y)
        gini_right = self._gini_index(right_y)
        n = len(y)
        child_gini = (len(left_y) / n) * gini_left + (len(right_y) / n) * gini_right
        return parent_gini - child_gini

    def _majority_error(self, y, left_y, right_y):
        parent_error = self._majority_error_value(y)
        majority_error_left = self._majority_error_value(left_y)
        majority_error_right = self._majority_error_value(right_y)
        n = len(y)
        child_error = (len(left_y) / n) * majority_error_left + (len(right_y) / n) * majority_error_right
        return parent_error - child_error

    def _split(self, X, feat, value):
        if self.feature_types[feat] == "numerical":
            left_idxs = np.where(X[:, feat].astype(float) <= value)[0]
            right_idxs = np.where(X[:, feat].astype(float) > value)[0]
        else:
            left_idxs = np.where(X[:, feat] == value)[0]
            right_idxs = np.where(X[:, feat] != value)[0]
        return left_idxs, right_idxs

    def _entropy(self, y):
        label_counts = Counter(y)
        total = len(y)
        entropy_value = 0.0

        for count in label_counts.values():
            p = count / total
            if p > 0:
                entropy_value -= p * np.log2(p)

        return entropy_value

    def _gini_index(self, y):
        counts = Counter(y)
        total = len(y)
        return 1.0 - sum((count / total) ** 2 for count in counts.values())

    def _majority_error_value(self, y):
        counts = Counter(y)
        total = len(y)
        return 1 - max(counts.values()) / total

    def _most_common_label(self, y):
        return Counter(y).most_common(1)[0][0]

    def predict(self, X):
        X = self._replace_missing_values(X)
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if self.feature_types[node.feature] == "numerical":
            if x[node.feature] <= node.threshold:
                return self._traverse_tree(x, node.left)
            else:
                return self._traverse_tree(x, node.right)
        else:
            if x[node.feature] == node.threshold:
                return self._traverse_tree(x, node.left)
            else:
                return self._traverse_tree(x, node.right)

column_headers = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'label']

df_train = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", names=column_headers)
df_test = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", names=column_headers)

X_train = df_train.drop('label', axis=1).values
y_train = df_train['label'].values
X_test = df_test.drop('label', axis=1).values
y_test = df_test['label'].values

metrics = {crit: {'train': [], 'test': []} for crit in ['information_gain', 'majority_error', 'gini']}
best_depths = {crit: 0 for crit in metrics.keys()}
best_accs = {crit: 0 for crit in metrics.keys()}

max_depth = int(input("Enter maximum tree depth: "))

for depth in range(1, max_depth + 1):
    for criterion in metrics.keys():
        clf = DecisionTree(max_depth=depth, criterion=criterion)
        clf.fit(X_train, y_train)

        y_train_pred = clf.predict(X_train)
        y_test_pred = clf.predict(X_test)

        train_acc = np.mean(y_train == y_train_pred)
        test_acc = np.mean(y_test == y_test_pred)

        metrics[criterion]['train'].append(round(1 - train_acc, 3))
        metrics[criterion]['test'].append(round(1 - test_acc, 3))

        if test_acc > best_accs[criterion]:
            best_accs[criterion] = test_acc
            best_depths[criterion] = depth

table_data = []
for depth in range(1, len(metrics['information_gain']['train']) + 1):
    row = [depth,
           metrics['information_gain']['train'][depth - 1],
           metrics['information_gain']['test'][depth - 1],
           metrics['majority_error']['train'][depth - 1],
           metrics['majority_error']['test'][depth - 1],
           metrics['gini']['train'][depth - 1],
           metrics['gini']['test'][depth - 1]]

    if depth == best_depths['information_gain']:
        row = ['*' + str(depth),
               '*' + str(metrics['information_gain']['train'][depth - 1]),
               '*' + str(metrics['information_gain']['test'][depth - 1]),
               str(metrics['majority_error']['train'][depth - 1]),
               str(metrics['majority_error']['test'][depth - 1]),
               str(metrics['gini']['train'][depth - 1]),
               str(metrics['gini']['test'][depth - 1])]
    elif depth == best_depths['majority_error']:
        row = [str(depth),
               str(metrics['information_gain']['train'][depth - 1]),
               str(metrics['information_gain']['test'][depth - 1]),
               '*' + str(metrics['majority_error']['train'][depth - 1]),
               '*' + str(metrics['majority_error']['test'][depth - 1]),
               str(metrics['gini']['train'][depth - 1]),
               str(metrics['gini']['test'][depth - 1])]
    elif depth == best_depths['gini']:
        row = [str(depth),
               str(metrics['information_gain']['train'][depth - 1]),
               str(metrics['information_gain']['test'][depth - 1]),
               str(metrics['majority_error']['train'][depth - 1]),
               str(metrics['majority_error']['test'][depth - 1]),
               '*' + str(metrics['gini']['train'][depth - 1]),
               '*' + str(metrics['gini']['test'][depth - 1])]

    table_data.append(row)

headers = ["Depth", "I.G(Train)", "I.G(Test)", "M.E(Train)", "M.E(Test)", "Gini(Train)", "Gini(Test)"]
print(tabulate(table_data, headers=headers, tablefmt="pretty"))

for crit in ['information_gain', 'majority_error', 'gini']:
    print(f"Least {crit.replace('_', ' ').title()} error observed at depth {best_depths[crit]} with test loss {1-best_accs[crit]:.3f}.")

Enter maximum tree depth: 16
+-------+------------+-----------+------------+-----------+-------------+------------+
| Depth | I.G(Train) | I.G(Test) | M.E(Train) | M.E(Test) | Gini(Train) | Gini(Test) |
+-------+------------+-----------+------------+-----------+-------------+------------+
|   1   |   0.119    |   0.119   |   0.109    |   0.109   |    0.109    |   0.109    |
|   2   |   0.106    |   0.106   |   0.108    |   0.108   |    0.109    |   0.109    |
|   3   |   0.105    |   0.105   |   0.107    |   0.107   |    0.106    |   0.106    |
|   4   |   0.104    |   0.104   |   0.105    |   0.105   |    0.104    |   0.104    |
|   5   |   0.099    |   0.099   |   0.102    |   0.102   |    0.099    |   0.099    |
|   6   |   0.093    |   0.093   |   0.099    |   0.099   |    0.091    |   0.091    |
|   7   |   0.088    |   0.088   |   0.098    |   0.098   |    0.086    |   0.086    |
|   8   |   0.082    |   0.082   |   0.097    |   0.097   |    0.081    |   0.081    |
|   9   |   0.

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter

def entropy(y):
    label_counts = Counter(y)
    total = len(y)
    entropy_value = 0.0
    for count in label_counts.values():
        p = count / total
        if p > 0:
            entropy_value -= p * np.log2(p)
    return entropy_value

def gini_index(y):
    counts = Counter(y)
    total = len(y)
    return 1.0 - sum((count / total) ** 2 for count in counts.values())

def majority_error(y):
    counts = Counter(y)
    total = len(y)
    return 1 - max(counts.values()) / total

def weighted_majority_error(y, left_y, right_y):
    parent_error = majority_error(y)
    left_error = majority_error(left_y)
    right_error = majority_error(right_y)
    return (len(left_y) / len(y)) * left_error + (len(right_y) / len(y)) * right_error - parent_error

def weighted_gini(y, left_y, right_y):
    parent_gini = gini_index(y)
    left_gini = gini_index(left_y)
    right_gini = gini_index(right_y)
    return (len(left_y) / len(y)) * left_gini + (len(right_y) / len(y)) * right_gini - parent_gini

def select_best_attribute(X, y):
    best_criteria = -np.inf
    best_attribute, best_threshold = None, None
    n_features = X.shape[1]

    for attribute in range(n_features):
        if np.issubdtype(X[:, attribute].dtype, np.number):  # Numerical
            threshold = np.median(X[:, attribute])
            left_indices = np.where(X[:, attribute] <= threshold)[0]
            right_indices = np.where(X[:, attribute] > threshold)[0]
            if len(left_indices) == 0 or len(right_indices) == 0:
                continue
            criteria_value = information_gain(y, y[left_indices], y[right_indices])
            if criteria_value > best_criteria:
                best_criteria = criteria_value
                best_attribute = attribute
                best_threshold = threshold
        else:  # Categorical
            for val in np.unique(X[:, attribute]):
                left_indices, right_indices = split(X, attribute, val)
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                criteria_value = information_gain(y, y[left_indices], y[right_indices])
                if criteria_value > best_criteria:
                    best_criteria = criteria_value
                    best_attribute = attribute
                    best_threshold = val

    return best_attribute, best_threshold

def split(X, attribute, value):
    if np.issubdtype(X[:, attribute].dtype, np.number):  # Numerical
        left_indices = np.where(X[:, attribute] <= value)[0]
        right_indices = np.where(X[:, attribute] > value)[0]
    else:  # Categorical
        left_indices = np.where(X[:, attribute] == value)[0]
        right_indices = np.where(X[:, attribute] != value)[0]
    return left_indices, right_indices

def information_gain(y, left_y, right_y):
    parent_entropy = entropy(y)
    left_entropy = entropy(left_y)
    right_entropy = entropy(right_y)
    child_entropy = (len(left_y) / len(y)) * left_entropy + (len(right_y) / len(y)) * right_entropy
    return parent_entropy - child_entropy

def id3(X, y, depth=0, max_depth=100):
    n_samples, n_labels = len(y), len(np.unique(y))

    if depth >= max_depth or n_labels == 1:
        return majority_vote(y)

    best_attribute, best_threshold = select_best_attribute(X, y)

    if best_attribute is None:
        return majority_vote(y)

    left_indices, right_indices = split(X, best_attribute, best_threshold)

    if len(left_indices) == 0 or len(right_indices) == 0:
        return majority_vote(y)

    left_node = id3(X[left_indices], y[left_indices], depth + 1, max_depth)
    right_node = id3(X[right_indices], y[right_indices], depth + 1, max_depth)

    return (best_attribute, best_threshold, left_node, right_node)

def majority_vote(y):
    return Counter(y).most_common(1)[0][0]

def predict(x, tree):
    if isinstance(tree, tuple):
        attribute, threshold, left_node, right_node = tree
        if x[attribute] <= threshold:
            return predict(x, left_node)
        else:
            return predict(x, right_node)
    else:
        return tree

def calculate_error(y_true, y_pred):
    return np.mean(y_true != y_pred)

def run_decision_tree(X_train, y_train, X_test, y_test, max_depth=100):
    depths = range(1, 17)
    train_ig, test_ig, train_me, test_me, train_gini, test_gini = [], [], [], [], [], []

    for depth in depths:
        # Information Gain
        tree_ig = id3(X_train, y_train, max_depth=depth)
        y_train_pred_ig = np.array([predict(x, tree_ig) for x in X_train])
        y_test_pred_ig = np.array([predict(x, tree_ig) for x in X_test])

        train_ig.append(calculate_error(y_train, y_train_pred_ig))
        test_ig.append(calculate_error(y_test, y_test_pred_ig))

        # Majority Error
        tree_me = id3(X_train, y_train, max_depth=depth)
        y_train_pred_me = np.array([predict(x, tree_me) for x in X_train])
        y_test_pred_me = np.array([predict(x, tree_me) for x in X_test])

        train_me.append(calculate_error(y_train, y_train_pred_me))
        test_me.append(calculate_error(y_test, y_test_pred_me))

        # Gini Index
        tree_gini = id3(X_train, y_train, max_depth=depth)
        y_train_pred_gini = np.array([predict(x, tree_gini) for x in X_train])
        y_test_pred_gini = np.array([predict(x, tree_gini) for x in X_test])

        train_gini.append(calculate_error(y_train, y_train_pred_gini))
        test_gini.append(calculate_error(y_test, y_test_pred_gini))

    # Output the results
    print(f"\n{'Depth':<6} {'I.G(Train)':<12} {'I.G(Test)':<12} {'M.E(Train)':<12} {'M.E(Test)':<12} {'Gini(Train)':<12} {'Gini(Test)':<12}")
    for i in range(len(depths)):
        print(f"{depths[i]:<6} {train_ig[i]:<12.3f} {test_ig[i]:<12.3f} {train_me[i]:<12.3f} {test_me[i]:<12.3f} {train_gini[i]:<12.3f} {test_gini[i]:<12.3f}")

    # Find minimum test error for each criterion
    min_ig_test = min(test_ig)
    min_me_test = min(test_me)
    min_gini_test = min(test_gini)

    print(f"\nLeast Information Gain test error observed at depth {test_ig.index(min_ig_test) + 1} with error {min_ig_test:.3f}.")
    print(f"Least Majority Error test error observed at depth {test_me.index(min_me_test) + 1} with error {min_me_test:.3f}.")
    print(f"Least Gini test error observed at depth {test_gini.index(min_gini_test) + 1} with error {min_gini_test:.3f}.")

# Load dataset
column_headers = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan',
                  'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'label']

df_train = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", names=column_headers)
df_test = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/test.csv", names=column_headers)

X_train = df_train.drop('label', axis=1).values
y_train = df_train['label'].values
X_test = df_test.drop('label', axis=1).values
y_test = df_test['label'].values

# Run the decision tree experiment
run_decision_tree(X_train, y_train, X_test, y_test)



Depth  I.G(Train)   I.G(Test)    M.E(Train)   M.E(Test)    Gini(Train)  Gini(Test)  
1      0.215        0.220        0.215        0.220        0.215        0.220       
2      0.215        0.220        0.215        0.220        0.215        0.220       
3      0.883        0.878        0.883        0.878        0.883        0.878       
4      0.886        0.879        0.886        0.879        0.886        0.879       
5      0.886        0.879        0.886        0.879        0.886        0.879       
6      0.886        0.879        0.886        0.879        0.886        0.879       
7      0.886        0.879        0.886        0.879        0.886        0.879       
8      0.885        0.879        0.885        0.879        0.885        0.879       
9      0.883        0.878        0.883        0.878        0.883        0.878       
10     0.883        0.879        0.883        0.879        0.883        0.879       
11     0.882        0.878        0.882        0.878        0.882

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
from tabulate import tabulate

def entropy(y):
    label_counts = Counter(y)
    total = len(y)
    entropy_value = 0.0
    for count in label_counts.values():
        p = count / total
        if p > 0:
            entropy_value -= p * np.log2(p)
    return entropy_value

def gini_index(y):
    counts = Counter(y)
    total = len(y)
    return 1.0 - sum((count / total) ** 2 for count in counts.values())

def majority_error(y):
    counts = Counter(y)
    total = len(y)
    return 1 - max(counts.values()) / total

def split(X, attribute, value):
    if np.issubdtype(X[:, attribute].dtype, np.number):  # Numerical
        left_indices = np.where(X[:, attribute] <= value)[0]
        right_indices = np.where(X[:, attribute] > value)[0]
    else:  # Categorical
        left_indices = np.where(X[:, attribute] == value)[0]
        right_indices = np.where(X[:, attribute] != value)[0]
    return left_indices, right_indices

def calculate_criteria_value(y, criterion):
    if criterion == 'information_gain':
        return entropy(y)
    elif criterion == 'gini':
        return gini_index(y)
    elif criterion == 'majority_error':
        return majority_error(y)

def calculate_gain(y, left_y, right_y, criterion):
    n = len(y)
    parent_value = calculate_criteria_value(y, criterion)
    left_value = calculate_criteria_value(left_y, criterion)
    right_value = calculate_criteria_value(right_y, criterion)
    child_value = (len(left_y) / n) * left_value + (len(right_y) / n) * right_value
    return parent_value - child_value

def select_best_split(X, y, criterion):
    best_gain = -np.inf
    best_attribute = None
    best_threshold = None
    n_features = X.shape[1]

    for attribute in range(n_features):
        if np.issubdtype(X[:, attribute].dtype, np.number):  # Numerical
            thresholds = np.unique(X[:, attribute])
            for threshold in thresholds:
                left_indices, right_indices = split(X, attribute, threshold)
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                gain = calculate_gain(y, y[left_indices], y[right_indices], criterion)
                if gain > best_gain:
                    best_gain = gain
                    best_attribute = attribute
                    best_threshold = threshold
        else:  # Categorical
            unique_values = np.unique(X[:, attribute])
            for value in unique_values:
                left_indices, right_indices = split(X, attribute, value)
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                gain = calculate_gain(y, y[left_indices], y[right_indices], criterion)
                if gain > best_gain:
                    best_gain = gain
                    best_attribute = attribute
                    best_threshold = value

    return best_attribute, best_threshold

def majority_vote(y):
    return Counter(y).most_common(1)[0][0]

def id3(X, y, depth=0, max_depth=100, criterion='information_gain'):
    n_samples, n_labels = len(y), len(np.unique(y))

    if depth >= max_depth or n_labels == 1:
        return majority_vote(y)

    best_attribute, best_threshold = select_best_split(X, y, criterion)

    if best_attribute is None:
        return majority_vote(y)

    left_indices, right_indices = split(X, best_attribute, best_threshold)

    if len(left_indices) == 0 or len(right_indices) == 0:
        return majority_vote(y)

    left_node = id3(X[left_indices], y[left_indices], depth + 1, max_depth, criterion)
    right_node = id3(X[right_indices], y[right_indices], depth + 1, max_depth, criterion)

    return (best_attribute, best_threshold, left_node, right_node)

def predict(x, tree):
    if isinstance(tree, tuple):
        attribute, threshold, left_node, right_node = tree
        if x[attribute] <= threshold:
            return predict(x, left_node)
        else:
            return predict(x, right_node)
    else:
        return tree

def calculate_error(y_true, y_pred):
    return np.mean(y_true != y_pred)

def run_decision_tree(X_train, y_train, X_test, y_test, max_depth):
    metrics = {criterion: {'train': [], 'test': []} for criterion in ['information_gain', 'majority_error', 'gini']}

    for depth in range(1, max_depth + 1):
        for criterion in metrics.keys():
            tree = id3(X_train, y_train, max_depth=depth, criterion=criterion)
            y_train_pred = np.array([predict(x, tree) for x in X_train])
            y_test_pred = np.array([predict(x, tree) for x in X_test])

            train_acc = np.mean(y_train == y_train_pred)
            test_acc = np.mean(y_test == y_test_pred)

            metrics[criterion]['train'].append(round(1 - train_acc, 3))
            metrics[criterion]['test'].append(round(1 - test_acc, 3))

    # Output the results
    table_data = []
    for depth in range(1, len(metrics['information_gain']['train']) + 1):
        row = [depth,
               metrics['information_gain']['train'][depth - 1],
               metrics['information_gain']['test'][depth - 1],
               metrics['majority_error']['train'][depth - 1],
               metrics['majority_error']['test'][depth - 1],
               metrics['gini']['train'][depth - 1],
               metrics['gini']['test'][depth - 1]]

        table_data.append(row)

    headers = ["Depth", "I.G(Train)", "I.G(Test)", "M.E(Train)", "M.E(Test)", "Gini(Train)", "Gini(Test)"]
    print(tabulate(table_data, headers=headers, tablefmt="pretty"))

    for crit in ['information_gain', 'majority_error', 'gini']:
        print(f"Least {crit.replace('_', ' ').title()} error observed at depth {metrics[crit]['test'].index(min(metrics[crit]['test'])) + 1} with error {min(metrics[crit]['test']):.3f}.")

# Load dataset
column_headers = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan',
                  'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'label']

df_train = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", names=column_headers)
df_test = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/test.csv", names=column_headers)

X_train = df_train.drop('label', axis=1).values
y_train = df_train['label'].values
X_test = df_test.drop('label', axis=1).values
y_test = df_test['label'].values

# Run the decision tree experiment
max_depth = 16  # or adjust based on your needs
run_decision_tree(X_train, y_train, X_test, y_test, max_depth)


+-------+------------+-----------+------------+-----------+-------------+------------+
| Depth | I.G(Train) | I.G(Test) | M.E(Train) | M.E(Test) | Gini(Train) | Gini(Test) |
+-------+------------+-----------+------------+-----------+-------------+------------+
|   1   |   0.215    |   0.22    |   0.215    |   0.22    |    0.215    |    0.22    |
|   2   |   0.215    |   0.22    |   0.336    |   0.345   |    0.13     |   0.131    |
|   3   |   0.883    |   0.878   |   0.783    |   0.78    |    0.787    |   0.782    |
|   4   |   0.886    |   0.879   |    0.78    |   0.779   |    0.79     |   0.783    |
|   5   |   0.886    |   0.879   |   0.771    |   0.774   |    0.791    |   0.783    |
|   6   |   0.886    |   0.879   |   0.723    |   0.719   |    0.789    |   0.782    |
|   7   |   0.886    |   0.879   |   0.699    |   0.693   |    0.789    |   0.782    |
|   8   |   0.885    |   0.879   |   0.697    |   0.692   |    0.788    |   0.782    |
|   9   |   0.883    |   0.878   |   0.695 

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
from tabulate import tabulate

def create_node(value=None):
    """Creates a tree node."""
    return {'value': value, 'left': None, 'right': None}

def is_leaf_node(node):
    """Checks if the node is a leaf node."""
    return 'value' in node

def determine_feature_types(X):
    """Determines the feature types of the dataset."""
    feature_types = []
    for i in range(X.shape[1]):
        unique_values = np.unique(X[:, i])
        if isinstance(unique_values[0], (int, float)) and len(unique_values) > 10:
            feature_types.append("numerical")
        else:
            feature_types.append("categorical")
    return feature_types

def split_data(X, feature, threshold, feature_types):
    """Splits the data into left and right subsets based on the threshold."""
    if feature_types[feature] == "numerical":
        left_indices = np.where(X[:, feature] <= threshold)[0]
        right_indices = np.where(X[:, feature] > threshold)[0]
    else:
        left_indices = np.where(X[:, feature] == threshold)[0]
        right_indices = np.where(X[:, feature] != threshold)[0]
    return left_indices, right_indices

def calculate_entropy(y):
    """Calculates the entropy of a label array."""
    label_counts = Counter(y)
    total = len(y)
    entropy_value = 0.0
    for count in label_counts.values():
        p = count / total
        if p > 0:
            entropy_value -= p * np.log2(p)
    return entropy_value

def calculate_gini_index(y):
    """Calculates the Gini index of a label array."""
    counts = Counter(y)
    total = len(y)
    return 1.0 - sum((count / total) ** 2 for count in counts.values())

def calculate_majority_error(y):
    """Calculates the majority error of a label array."""
    counts = Counter(y)
    total = len(y)
    return 1 - max(counts.values()) / total

def information_gain(y, left_y, right_y):
    """Calculates the information gain from a split."""
    n = len(y)
    parent_entropy = calculate_entropy(y)
    left_entropy = calculate_entropy(left_y)
    right_entropy = calculate_entropy(right_y)
    child_entropy = (len(left_y) / n) * left_entropy + (len(right_y) / n) * right_entropy
    return parent_entropy - child_entropy

def best_split(X, y, feature_types):
    """Finds the best feature and threshold to split on."""
    best_gain = -np.inf
    best_feature = None
    best_threshold = None
    n_features = X.shape[1]

    for feature in range(n_features):
        if feature_types[feature] == "numerical":
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_indices, right_indices = split_data(X, feature, threshold, feature_types)
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                gain = information_gain(y, y[left_indices], y[right_indices])
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        else:
            unique_values = np.unique(X[:, feature])
            for value in unique_values:
                left_indices, right_indices = split_data(X, feature, value, feature_types)
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                gain = information_gain(y, y[left_indices], y[right_indices])
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = value

    return best_feature, best_threshold

def id3(X, y, depth=0, max_depth=100):
    """Builds the decision tree using the ID3 algorithm."""
    n_samples, n_labels = len(y), len(np.unique(y))

    if depth >= max_depth or n_labels == 1:
        return create_node(value=Counter(y).most_common(1)[0][0])

    feature_types = determine_feature_types(X)
    best_feature, best_threshold = best_split(X, y, feature_types)

    if best_feature is None:
        return create_node(value=Counter(y).most_common(1)[0][0])

    left_indices, right_indices = split_data(X, best_feature, best_threshold, feature_types)

    if len(left_indices) == 0 or len(right_indices) == 0:
        return create_node(value=Counter(y).most_common(1)[0][0])

    left_node = id3(X[left_indices], y[left_indices], depth + 1, max_depth)
    right_node = id3(X[right_indices], y[right_indices], depth + 1, max_depth)

    return {
        'feature': best_feature,
        'threshold': best_threshold,
        'left': left_node,
        'right': right_node
    }

def predict(x, tree):
    """Predicts the label for a single instance."""
    if is_leaf_node(tree):
        return tree['value']

    if x[tree['feature']] <= tree['threshold']:
        return predict(x, tree['left'])
    else:
        return predict(x, tree['right'])

def calculate_error(y_true, y_pred):
    """Calculates the error rate."""
    return np.mean(y_true != y_pred)

def run_decision_tree(X_train, y_train, X_test, y_test, max_depth):
    """Runs the decision tree algorithm and prints the results."""
    metrics = {criterion: {'train': [], 'test': []} for criterion in ['information_gain', 'majority_error', 'gini']}

    for depth in range(1, max_depth + 1):
        for criterion in metrics.keys():
            tree = id3(X_train, y_train, max_depth=depth)
            y_train_pred = np.array([predict(x, tree) for x in X_train])
            y_test_pred = np.array([predict(x, tree) for x in X_test])

            train_acc = np.mean(y_train == y_train_pred)
            test_acc = np.mean(y_test == y_test_pred)

            metrics[criterion]['train'].append(round(1 - train_acc, 3))
            metrics[criterion]['test'].append(round(1 - test_acc, 3))

    # Output the results
    table_data = []
    for depth in range(1, len(metrics['information_gain']['train']) + 1):
        row = [depth,
               metrics['information_gain']['train'][depth - 1],
               metrics['information_gain']['test'][depth - 1],
               metrics['majority_error']['train'][depth - 1],
               metrics['majority_error']['test'][depth - 1],
               metrics['gini']['train'][depth - 1],
               metrics['gini']['test'][depth - 1]]
        table_data.append(row)

    headers = ["Depth", "I.G(Train)", "I.G(Test)", "M.E(Train)", "M.E(Test)", "Gini(Train)", "Gini(Test)"]
    print(tabulate(table_data, headers=headers, tablefmt="pretty"))

    for crit in ['information_gain', 'majority_error', 'gini']:
        print(f"Least {crit.replace('_', ' ').title()} error observed at depth {metrics[crit]['test'].index(min(metrics[crit]['test'])) + 1} with error {min(metrics[crit]['test']):.3f}.")

# Load dataset
column_headers = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan',
                  'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'label']

df_train = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", names=column_headers)
df_test = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/test.csv", names=column_headers)

X_train = df_train.drop('label', axis=1).values
y_train = df_train['label'].values
X_test = df_test.drop('label', axis=1).values
y_test = df_test['label'].values

# Run the decision tree experiment
max_depth = 16
run_decision_tree(X_train, y_train, X_test, y_test, max_depth)


+-------+------------+-----------+------------+-----------+-------------+------------+
| Depth | I.G(Train) | I.G(Test) | M.E(Train) | M.E(Test) | Gini(Train) | Gini(Test) |
+-------+------------+-----------+------------+-----------+-------------+------------+
|   1   |   0.119    |   0.125   |   0.119    |   0.125   |    0.119    |   0.125    |
|   2   |   0.212    |   0.219   |   0.212    |   0.219   |    0.212    |   0.219    |
|   3   |   0.156    |   0.162   |   0.156    |   0.162   |    0.156    |   0.162    |
|   4   |   0.187    |   0.203   |   0.187    |   0.203   |    0.187    |   0.203    |
|   5   |   0.197    |   0.208   |   0.197    |   0.208   |    0.197    |   0.208    |
|   6   |   0.204    |   0.22    |   0.204    |   0.22    |    0.204    |    0.22    |
|   7   |   0.227    |   0.248   |   0.227    |   0.248   |    0.227    |   0.248    |
|   8   |   0.225    |   0.247   |   0.225    |   0.247   |    0.225    |   0.247    |
|   9   |   0.223    |   0.247   |   0.223 

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter

def create_node_bank(value=None):
    return {'value': value, 'left': None, 'right': None, 'feature': None, 'threshold': None}

def is_leaf_node_bank(node):
    return node['value'] is not None

def determine_feature_types_bank(X):
    feature_types = []
    for i in range(X.shape[1]):
        unique_values = np.unique(X[:, i])
        if isinstance(unique_values[0], (int, float)) and len(unique_values) > 10:
            feature_types.append("numerical")
        else:
            feature_types.append("categorical")
    return feature_types

def split_data_bank(X, feature, threshold):
    left_indices = np.where(X[:, feature] <= threshold)[0]
    right_indices = np.where(X[:, feature] > threshold)[0]
    return left_indices, right_indices

def calculate_entropy_bank(y):
    label_counts = Counter(y)
    total = len(y)
    entropy_value = 0.0
    for count in label_counts.values():
        p = count / total
        if p > 0:
            entropy_value -= p * np.log2(p)
    return entropy_value

def calculate_gini_index_bank(y):
    counts = Counter(y)
    total = len(y)
    return 1.0 - sum((count / total) ** 2 for count in counts.values())

def calculate_majority_error_bank(y):
    counts = Counter(y)
    total = len(y)
    return 1 - max(counts.values()) / total

def information_gain_bank(y, left_y, right_y):
    n = len(y)
    parent_entropy = calculate_entropy_bank(y)
    left_entropy = calculate_entropy_bank(left_y)
    right_entropy = calculate_entropy_bank(right_y)
    child_entropy = (len(left_y) / n) * left_entropy + (len(right_y) / n) * right_entropy
    return parent_entropy - child_entropy

def best_split_bank(X, y, feature_types):
    best_gain = -np.inf
    best_feature = None
    best_threshold = None
    n_features = X.shape[1]

    for feature in range(n_features):
        if feature_types[feature] == "numerical":
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_indices, right_indices = split_data_bank(X, feature, threshold)
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                gain = information_gain_bank(y, y[left_indices], y[right_indices])
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        else:
            unique_values = np.unique(X[:, feature])
            for value in unique_values:
                left_indices, right_indices = split_data_bank(X, feature, value)
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                gain = information_gain_bank(y, y[left_indices], y[right_indices])
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = value

    return best_feature, best_threshold

def build_tree_bank(X, y, depth=0, max_depth=100):
    n_samples, n_labels = len(y), len(np.unique(y))

    if depth >= max_depth or n_labels == 1:
        return create_node_bank(value=Counter(y).most_common(1)[0][0])

    feature_types = determine_feature_types_bank(X)
    best_feature, best_threshold = best_split_bank(X, y, feature_types)

    if best_feature is None:
        return create_node(value=Counter(y).most_common(1)[0][0])

    left_indices, right_indices = split_data_bank(X, best_feature, best_threshold)

    if len(left_indices) == 0 or len(right_indices) == 0:
        return create_node(value=Counter(y).most_common(1)[0][0])

    left_node = build_tree_bank(X[left_indices], y[left_indices], depth + 1, max_depth)
    right_node = build_tree_bank(X[right_indices], y[right_indices], depth + 1, max_depth)

    parent_node = create_node_bank()
    parent_node['feature'] = best_feature
    parent_node['threshold'] = best_threshold
    parent_node['left'] = left_node
    parent_node['right'] = right_node

    return parent_node

def predict_bank(instance, tree):
    if is_leaf_node_bank(tree):
        return tree['value']
    if instance[tree['feature']] <= tree['threshold']:
        return predict_bank(instance, tree['left'])
    else:
        return predict_bank(instance, tree['right'])

def run_decision_tree_bank(X_train, y_train, X_test, y_test, max_depth):
    metrics = {criterion: {'train': [], 'test': []} for criterion in ['information_gain', 'majority_error', 'gini']}

    for depth in range(1, max_depth + 1):
        for criterion in metrics.keys():
            tree = build_tree_bank(X_train, y_train, max_depth=depth)
            y_train_pred = np.array([predict_bank(x, tree) for x in X_train])
            y_test_pred = np.array([predict_bank(x, tree) for x in X_test])

            train_accuracy = np.mean(y_train == y_train_pred)
            test_accuracy = np.mean(y_test == y_test_pred)

            metrics[criterion]['train'].append(round(1 - train_accuracy, 3))
            metrics[criterion]['test'].append(round(1 - test_accuracy, 3))

    table_data = []
    for depth in range(1, max_depth + 1):
        row = [depth,
               metrics['information_gain']['train'][depth - 1],
               metrics['information_gain']['test'][depth - 1],
               metrics['majority_error']['train'][depth - 1],
               metrics['majority_error']['test'][depth - 1],
               metrics['gini']['train'][depth - 1],
               metrics['gini']['test'][depth - 1]]
        table_data.append(row)

    print("\nResults Summary:")
    print(f"{'Depth':<6} {'I.G(Train)':<12} {'I.G(Test)':<12} {'M.E(Train)':<12} {'M.E(Test)':<12} {'Gini(Train)':<12} {'Gini(Test)':<12}")
    for depth in range(1, max_depth + 1):
        print(f"{depth:<6} {metrics['information_gain']['train'][depth - 1]:<12} {metrics['information_gain']['test'][depth - 1]:<12} "
              f"{metrics['majority_error']['train'][depth - 1]:<12} {metrics['majority_error']['test'][depth - 1]:<12} "
              f"{metrics['gini']['train'][depth - 1]:<12} {metrics['gini']['test'][depth - 1]:<12}")

    for crit in ['information_gain', 'majority_error', 'gini']:
        print(f"Least {crit.replace('_', ' ').title()} error observed at depth {metrics[crit]['test'].index(min(metrics[crit]['test'])) + 1} with error {min(metrics[crit]['test']):.3f}.")

def experiment():
    column_headers_bank = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan',
                  'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'label']

    df_train_bank = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", header=None, names=column_headers_bank)
    df_test_bank = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/test.csv", header=None, names=column_headers_bank)

    X_train_bank = df_train_bank.drop('label', axis=1).values
    y_train_bank = df_train_bank['label'].values
    X_test_bank = df_test_bank.drop('label', axis=1).values
    y_test_bank = df_test_bank['label'].values

    max_depth = 16

    run_decision_tree_bank(X_train_bank, y_train_bank, X_test_bank, y_test_bank, max_depth)

if __name__ == "__main__":
    experiment()



Results Summary:
Depth  I.G(Train)   I.G(Test)    M.E(Train)   M.E(Test)    Gini(Train)  Gini(Test)  
1      0.119        0.125        0.119        0.125        0.119        0.125       
2      0.115        0.122        0.115        0.122        0.115        0.122       
3      0.108        0.118        0.108        0.118        0.108        0.118       
4      0.104        0.117        0.104        0.117        0.104        0.117       
5      0.1          0.118        0.1          0.118        0.1          0.118       
6      0.093        0.118        0.093        0.118        0.093        0.118       
7      0.083        0.119        0.083        0.119        0.083        0.119       
8      0.074        0.114        0.074        0.114        0.074        0.114       
9      0.061        0.117        0.061        0.117        0.061        0.117       
10     0.051        0.117        0.051        0.117        0.051        0.117       
11     0.041        0.125        0.041        0

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter

def create_leaf_bank(value):
    return {'value': value, 'left': None, 'right': None, 'feature': None, 'threshold': None}

def is_leaf_bank(node):
    return node['value'] is not None

def identify_feature_types_bank(X):
    feature_types = []
    for i in range(X.shape[1]):
        unique_values = np.unique(X[:, i])
        if isinstance(unique_values[0], (int, float)) and len(unique_values) > 10:
            feature_types.append("numerical")
        else:
            feature_types.append("categorical")
    return feature_types

def impute_missing_values_bank(X):
    majority_values = {}
    for i in range(X.shape[1]):
        if 'unknown' in X[:, i]:
            majority_value = Counter(X[X[:, i] != 'unknown', i]).most_common(1)[0][0]
            majority_values[i] = majority_value
    return majority_values

def replace_missing_values_bank(X, majority_values):
    X_imputed = X.copy()
    for i in majority_values:
        X_imputed[X[:, i] == 'unknown', i] = majority_values[i]
    return X_imputed

def split_dataset_bank(X, feature, threshold):
    left_indices = np.where(X[:, feature] <= threshold)[0]
    right_indices = np.where(X[:, feature] > threshold)[0]
    return left_indices, right_indices

def calculate_entropy_bank(y):
    label_counts = Counter(y)
    total = len(y)
    entropy = 0.0
    for count in label_counts.values():
        probability = count / total
        if probability > 0:
            entropy -= probability * np.log2(probability)
    return entropy

def calculate_gini_index_bank(y):
    counts = Counter(y)
    total = len(y)
    return 1.0 - sum((count / total) ** 2 for count in counts.values())

def calculate_majority_error_bank(y):
    counts = Counter(y)
    total = len(y)
    return 1 - max(counts.values()) / total

def information_gain_bank(y, left_y, right_y):
    n = len(y)
    if n == 0:
        return 0
    parent_entropy = calculate_entropy_bank(y)
    left_entropy = calculate_entropy_bank(left_y)
    right_entropy = calculate_entropy_bank(right_y)
    child_entropy = (len(left_y) / n) * left_entropy + (len(right_y) / n) * right_entropy
    return parent_entropy - child_entropy

def find_best_split_bank(X, y, feature_types):
    best_gain = -np.inf
    best_feature = None
    best_threshold = None
    num_features = X.shape[1]

    for feature in range(num_features):
        if feature_types[feature] == "numerical":
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_indices, right_indices = split_dataset_bank(X, feature, threshold)
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                gain = information_gain_bank(y, y[left_indices], y[right_indices])
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        else:
            unique_values = np.unique(X[:, feature])
            for value in unique_values:
                if value == 'unknown':
                    continue
                left_indices, right_indices = split_dataset_bank(X, feature, value)
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                gain = information_gain_bank(y, y[left_indices], y[right_indices])
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = value

    return best_feature, best_threshold

def build_tree_bank(X, y, depth=0, max_depth=100):
    n_samples, n_labels = len(y), len(np.unique(y))

    if depth >= max_depth or n_labels == 1:
        return create_leaf_bank(Counter(y).most_common(1)[0][0])

    feature_types = identify_feature_types_bank(X)
    best_feature, best_threshold = find_best_split_bank(X, y, feature_types)

    if best_feature is None:
        return create_leaf_bank(Counter(y).most_common(1)[0][0])

    left_indices, right_indices = split_dataset_bank(X, best_feature, best_threshold)

    if len(left_indices) == 0 or len(right_indices) == 0:
        return create_leaf_bank(Counter(y).most_common(1)[0][0])

    left_subtree = build_tree_bank(X[left_indices], y[left_indices], depth + 1, max_depth)
    right_subtree = build_tree_bank(X[right_indices], y[right_indices], depth + 1, max_depth)

    node = create_leaf_bank(None)
    node['feature'] = best_feature
    node['threshold'] = best_threshold
    node['left'] = left_subtree
    node['right'] = right_subtree

    return node

def traverse_tree_bank(instance, tree):
    if is_leaf_bank(tree):
        return tree['value']
    if instance[tree['feature']] <= tree['threshold']:
        return traverse_tree_bank(instance, tree['left'])
    else:
        return traverse_tree_bank(instance, tree['right'])

def run_decision_tree_bank(X_train, y_train, X_test, y_test, max_depth):
    metrics = {criterion: {'train': [], 'test': []} for criterion in ['information_gain', 'majority_error', 'gini']}
    best_depths = {criterion: 0 for criterion in metrics.keys()}
    best_accuracies = {criterion: 0 for criterion in metrics.keys()}

    majority_values = impute_missing_values_bank(X_train)
    X_train = replace_missing_values_bank(X_train, majority_values)
    X_test = replace_missing_values_bank(X_test, majority_values)

    for depth in range(1, max_depth + 1):
        for criterion in metrics.keys():
            tree = build_tree_bank(X_train, y_train, max_depth=depth)
            y_train_pred = np.array([traverse_tree_bank(x, tree) for x in X_train])
            y_test_pred = np.array([traverse_tree_bank(x, tree) for x in X_test])

            train_accuracy = np.mean(y_train == y_train_pred)
            test_accuracy = np.mean(y_test == y_test_pred)

            metrics[criterion]['train'].append(round(1 - train_accuracy, 3))
            metrics[criterion]['test'].append(round(1 - test_accuracy, 3))

            if test_accuracy > best_accuracies[criterion]:
                best_accuracies[criterion] = test_accuracy
                best_depths[criterion] = depth

    print("\nResults Summary:")
    print(f"{'Depth':<6} {'I.G(Train)':<12} {'I.G(Test)':<12} {'M.E(Train)':<12} {'M.E(Test)':<12} {'Gini(Train)':<12} {'Gini(Test)':<12}")
    for depth in range(1, max_depth + 1):
        print(f"{depth:<6} {metrics['information_gain']['train'][depth - 1]:<12} {metrics['information_gain']['test'][depth - 1]:<12} "
              f"{metrics['majority_error']['train'][depth - 1]:<12} {metrics['majority_error']['test'][depth - 1]:<12} "
              f"{metrics['gini']['train'][depth - 1]:<12} {metrics['gini']['test'][depth - 1]:<12}")

    for criteria in ['information_gain', 'majority_error', 'gini']:
        print(f"Least {criteria.replace('_', ' ').title()} error observed at depth {best_depths[criteria]} with error {1 - best_accuracies[criteria]:.3f}.")

def experiment():
    column_names_bank = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']
    column_headers_bank = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan',
                  'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'label']

    df_train_bank = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/train.csv", header=None, names=column_headers_bank)
    df_test_bank = pd.read_csv("/content/drive/My Drive/cs6350/assignments/assignment1/bank/test.csv", header=None, names=column_headers_bank)

    X_train_bank = df_train_bank.drop('label', axis=1).values
    y_train_bank = df_train_bank['label'].values
    X_test_bank = df_test_bank.drop('label', axis=1).values
    y_test_bank = df_test_bank['label'].values

    max_depth = 16

    run_decision_tree_bank(X_train_bank, y_train_bank, X_test_bank, y_test_bank, max_depth)

if __name__ == "__main__":
    experiment()


Results Summary:
Depth  I.G(Train)   I.G(Test)    M.E(Train)   M.E(Test)    Gini(Train)  Gini(Test)  
1      0.119        0.125        0.119        0.125        0.119        0.125       
2      0.11         0.12         0.11         0.12         0.11         0.12        
3      0.098        0.11         0.098        0.11         0.098        0.11        
4      0.094        0.112        0.094        0.112        0.094        0.112       
5      0.092        0.112        0.092        0.112        0.092        0.112       
6      0.083        0.113        0.083        0.113        0.083        0.113       
7      0.077        0.113        0.077        0.113        0.077        0.113       
8      0.069        0.12         0.069        0.12         0.069        0.12        
9      0.063        0.118        0.063        0.118        0.063        0.118       
10     0.053        0.123        0.053        0.123        0.053        0.123       
11     0.044        0.124        0.044        0